<a href="https://colab.research.google.com/github/elliot-drew/pdbmd2cd-colab/blob/main/pdbmd2cd_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PDBMD2CD - Predicting circular dichroism spectra from atomic coordinates using Colab

This notebook is an Colab implementation of the PDBMD2CD algorithm described here: https://academic.oup.com/nar/article/48/W1/W17/5826170

The original webserver version of this program is available here: https://pdbmd2cd.cryst.bbk.ac.uk/

We would encourage users who wish to do very large numbers of predictions (>1000 at a time) to do so using this Colab notebook or the standalone version of PDBMD2CD available here: TBC

## Usage

- Upload a zip file called "proteins.zip" containing all the PDB files of the structures you want to run predictions for.
  - Click the folder icon on the left hand side of the page
  - Right click in the panel that opens up, and click the "Upload" option
  - Browse to and select the zip file you created in the dialog that appears
- Once this file is uploaded, in the "Runtime" menu at the top of the page, click "Run all"
- A number of dependencies need to be downloaded and installed for the prediction to run - this can take a few minutes, so be patient.
- When the predictions are done, a dialog window will appear with the results in a zip file - you can open or save this to your computer.

Inside the results file are: 
- the predicted CD spectrum for each structure - if any of the PDB files you uploaded had multiple models a spectrum will have ben produced for each model.
- The average CD spectrum of all uploaded structures
- A log file containing information on the run, including the identities of any structures that failed during the prediction process


## Install Conda - must be python 3.6

In [1]:
%env PYTHONPATH=

!wget -c https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
!chmod +x Miniconda3-4.5.4-Linux-x86_64.sh
!bash ./Miniconda3-4.5.4-Linux-x86_64.sh -b -f -p /usr/local
!conda install -q -y --prefix /usr/local python=3.6
import sys
sys.path.append('/usr/local/lib/python3.6/site-packages')

env: PYTHONPATH=
--2021-12-14 09:57:44--  https://repo.anaconda.com/miniconda/Miniconda3-4.5.4-Linux-x86_64.sh
Resolving repo.anaconda.com (repo.anaconda.com)... 104.16.130.3, 104.16.131.3, 2606:4700::6810:8203, ...
Connecting to repo.anaconda.com (repo.anaconda.com)|104.16.130.3|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 58468498 (56M) [application/x-sh]
Saving to: ‘Miniconda3-4.5.4-Linux-x86_64.sh’

Miniconda3-4.5.4-Li 100%[===================>]  55.76M   183MB/s    in 0.3s    

2021-12-14 09:57:44 (183 MB/s) - ‘Miniconda3-4.5.4-Linux-x86_64.sh’ saved [58468498/58468498]

PREFIX=/usr/local
installing: python-3.6.5-hc3d631a_2 ...
Python 3.6.5 :: Anaconda, Inc.
installing: ca-certificates-2018.03.07-0 ...
installing: conda-env-2.6.0-h36134e3_1 ...
installing: libgcc-ng-7.2.0-hdf63c60_3 ...
installing: libstdcxx-ng-7.2.0-hdf63c60_3 ...
installing: libffi-3.2.1-hd88cf55_4 ...
installing: ncurses-6.1-hf484d3e_0 ...
installing: openssl-1.0.2o-h20670df_0 ...
i

## Install dssp using conda

In [2]:
 !conda install -y -q -c salilab dssp 

Solving environment: ...working... done

## Package Plan ##

  environment location: /usr/local

  added / updated specs: 
    - dssp


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    icu-58.2                   |       he6710b0_3        22.7 MB
    dssp-3.0.0                 |       hf484d3e_3         2.2 MB  salilab
    bzip2-1.0.8                |       h7b6447c_0         105 KB
    libboost-1.67.0            |       h46d08c1_4        20.9 MB
    ------------------------------------------------------------
                                           Total:        45.9 MB

The following NEW packages will be INSTALLED:

    bzip2:    1.0.8-h7b6447c_0         
    dssp:     3.0.0-hf484d3e_3  salilab
    icu:      58.2-he6710b0_3          
    libboost: 1.67.0-h46d08c1_4        

Preparing transaction: ...working... done
Verifying transaction: ...working... done
Executing transaction: ...wo

## check mkdssp executable is installed

In [ ]:
!which mkdssp

/usr/local/bin/mkdssp


## Install all the other requirements

In [3]:
!pip -q install scipy==1.3.1 networkx==2.3 biopython
!pip -q install plotly




     |████████████████████████████████| 25.2 MB 1.2 MB/s 
     |████████████████████████████████| 1.7 MB 44.2 MB/s 
     |████████████████████████████████| 2.3 MB 56.7 MB/s 
     |████████████████████████████████| 14.8 MB 23.1 MB/s 


## PDBMD2CD processing and predictions start here



In [4]:
#@title Make input and output directories. Unzip input files. { vertical-output: true, display-mode: "form" }
!mkdir -p ./in
!mkdir -p ./out

!unzip -q proteins.zip -d ./in

In [5]:
#@title Load in all the required reference information
# first the required text
ref_basis = [[0.04549958927582217,0.29966293348415524,0.08749579388553883,-0.021926078493204562,-0.026756051862796,-0.22605597016198464,0.015052411870521698],
[0.06473852039933865,0.5921380078464065,-0.045032153099206766,-0.012336612329034824,-0.286323306864428,-0.5792732004444402,0.14709198725259048],
[0.07668643657859799,0.6949017790222841,-0.0034046128249779622,-0.013690141264888933,-0.37103635626464226,-0.77912193052487,0.19312606462363352],
[0.0942753345176401,0.5642032277426066,0.07938139473405387,0.03485709356973305,-0.3678641651601304,-0.8413114298420321,0.1631901277856194],
[0.09518406721974121,0.697162858275815,0.14351963083624517,0.025039223187336992,-0.3868453004582004,-0.9092850086823343,0.15441467140941428],
[0.06990919926164298,0.5815299981130706,0.13840786421626272,0.004469570706741449,-0.2763253492122945,-0.5641098335697131,0.05816389853793963],
[0.06149239262804933,0.6244876213805535,0.1335224825874909,-0.026345682556166705,-0.24384883494849105,-0.5064837283655422,0.05770348945038217],
[0.04286608688273796,0.5618566808551639,0.02385936347027659,-0.00727399604445203,-0.2026698639814167,-0.32198330404889863,0.027282621246872474],
[0.027235254162842334,0.20798378519890542,0.010239727414396315,0.023460988189141507,-0.08877523814500021,-0.07864218647157764,-0.04272867638861619],
[0.028445800648038522,0.09072126077800789,0.06749451119796271,0.022032907401789634,-0.13983951056418,-0.11838114245599864,-0.037765793420621314],
[0.03419343506078483,0.43234027501361527,0.13853387997653233,-0.013639514068628199,-0.3796042835779824,-0.3427250608719564,0.003335596205007507],
[0.002095557130291208,0.5995695287317433,0.10119308512409732,-0.006257589422119472,-0.4994498771421959,-0.3947602817529675,0.01099904291182574],
[-0.04239795617931084,0.7399876295471228,0.09966265609258587,0.0029388042052555796,-0.513038307027302,-0.4787046756348522,0.01017271954996779],
[-0.12705277085121294,0.7811817064756409,0.046997035464835454,-0.01761591028468723,-0.40403895904478054,-0.3849049800348424,-0.007907524343746863],
[-0.20804079336782727,0.5379982349341913,0.0941950476356372,0.009298555777299315,-0.18662835575690673,-0.1993191983020964,-0.08901385243042848],
[-0.2804215409188777,0.39584551340141594,0.2467986979863022,-0.006263888033518149,-0.1563435759895114,-0.16711496784523897,-0.1407746489411689],
[-0.36767511823760424,0.6848506557259426,0.35267640207369816,-0.05346471135318859,-0.3590575646600172,-0.28985694043961124,-0.1348471276032634],
[-0.5107147112722247,0.9378437812128979,0.4075680199762345,-0.09955489218351492,-0.5435429652637904,-0.25000052304788195,-0.15150583215778313],
[-0.7156613102398469,0.8909261082392761,0.523073087923427,-0.1724678122855925,-0.6513975257575403,0.012888727147854412,-0.219264558534091],
[-0.967272567101261,1.2459526324140984,0.5800863947475171,-0.3424883182760236,-0.9144183534874272,-0.08948081978761735,-0.12941038536258764],
[-1.2415803892911241,1.2299146795839258,0.7698402018239996,-0.39112713974728425,-0.9887249616427729,-0.16316223785960904,-0.11815072258634718],
[-1.5656680443613773,0.9758296294730777,1.1604866623847854,-0.45206197549309807,-1.0559222914631337,-0.11202703774815478,-0.181841434209144],
[-1.9523005655546102,1.0568557811505583,1.5180974340243698,-0.5665193006648623,-1.292157813103672,-0.2088951811192395,-0.16503319773043146],
[-2.3921219113822625,0.8642896024213689,2.052728115088911,-0.6977894182378023,-1.3602631379856733,-0.15663564302559332,-0.23165624950477803],
[-2.897903557795404,0.7461637464459283,2.5613248152411696,-0.8640927656453958,-1.3789509606944832,-0.05516393120328367,-0.3119180781435631],
[-3.456645997577668,0.880396038142137,2.9231399458924234,-1.095614025279174,-1.5968979387062765,-0.17061021808211443,-0.27756034659988654],
[-4.07047420053331,0.7689678907458619,3.3158812213464888,-1.3226946903858567,-1.7184929601548924,-0.18657391585156163,-0.28295804976199446],
[-4.730138483946052,0.4550483694128121,3.8253812130728977,-1.5928683506831962,-1.8045098828863597,-0.04559490855757911,-0.364152765466735],
[-5.413682988545025,0.14291649848067267,4.397345626963742,-1.8343414481755236,-2.008162904062275,0.06117217724785265,-0.47306395466426826],
[-6.1136705369389865,-0.45009164081618724,5.026765109790392,-2.0592718851788074,-2.154886066891191,0.11856571827172913,-0.5748280395364675],
[-6.82063759656652,-1.1110724460026233,5.784588653153181,-2.2671623402779626,-2.1659084704405682,0.47293518181574556,-0.8317902016328511],
[-7.505207320991495,-1.8713161327145866,6.337891563176591,-2.439973142333016,-2.257253471847814,0.7839541142534587,-1.0251976284509037],
[-8.126394835671576,-2.873415690048354,6.890151337044296,-2.629063871549082,-2.256373232124901,1.2649599588500149,-1.2764750496921113],
[-8.673890671985875,-4.003731456682612,7.3075251689260154,-2.8309248502966824,-2.2646209684497007,1.7300873077123724,-1.5111035141619915],
[-9.135178271916903,-5.1824816422305835,7.452886980610049,-2.9816417282950933,-2.2596062070572938,2.176022278545213,-1.7238314311779779],
[-9.52562792765765,-6.199467031939459,7.324986708651077,-3.1163560863504856,-2.364578374234192,2.6249567671285297,-1.8782044160897557],
[-9.814226261184576,-6.765336067716861,6.99738219138271,-3.2448363969746428,-2.5859040380817184,3.011979195914271,-2.0088770737764197],
[-9.976825477931794,-7.280249579427883,6.583351279485366,-3.343562260070163,-2.7950751603449593,3.3033199087924507,-2.1233047975666235],
[-10.015827598410377,-7.754544295476965,6.230982298329559,-3.407604720566397,-2.9221505758613713,3.5060032678042177,-2.260987363755559],
[-9.97797561861588,-8.010044318334325,5.754824822981731,-3.4562771973500652,-3.073548086914253,3.5965525028007352,-2.3713797961612313],
[-9.83487941228355,-8.184082681298918,5.159979250508951,-3.387695294235373,-3.106623979417771,3.401549777366738,-2.4581093741359297],
[-9.644689241396035,-8.302086526310248,4.627817270079728,-3.2781167797469433,-3.1016559935658865,3.4586891128322095,-2.6488791397394067],
[-9.415417924348542,-7.909594510539142,3.80688452262949,-3.1287968518936475,-3.3480346960799094,3.0475523259171924,-2.6351814178284014],
[-9.183130068526227,-7.5988394816932106,3.088564820081525,-2.883763663246267,-3.3767220346779867,2.8927609829217222,-2.7503149576909562],
[-8.973912962622258,-7.292593670886273,2.472270955396823,-2.5616789694808118,-3.1508010310079246,2.682690564540794,-2.877852198366469],
[-8.80481899401209,-6.931802466284295,1.6648515602680451,-2.1176864485646867,-2.792021211631884,2.5027295668087897,-3.003442793110751],
[-8.699392349602586,-6.686462529818872,0.7035198643022431,-1.5575698275147598,-2.3124409095103555,2.1430907279951774,-3.042848625274843],
[-8.669496585823195,-6.581476290658427,-0.2577450411893534,-0.9241531150351594,-1.6304361498847464,2.1268437864677865,-3.223360214929796],
[-8.705005605320986,-6.4875697091715345,-1.4447179210842807,-0.23189220815900113,-1.0895283113370735,1.7235278175815196,-3.2138269716499437],
[-8.812318313907847,-6.345084729591878,-2.8158936052074868,0.6267342657383213,-0.4209711931652942,1.4125300947832993,-3.28415340785892],
[-8.89244398040731,-6.6390455503044095,-4.344111004259339,1.6279005198970593,0.38731225966229554,1.060241108869207,-3.349559119175895],
[-8.81297822463957,-7.261872198165825,-6.307655412754483,2.8148571687197594,1.0601174335801953,0.5661565495968452,-3.3231990113695344],
[-8.454858837659222,-8.177463841625116,-8.519790715464282,4.1118640382089415,1.7281171298260822,0.25265900866828694,-3.2994907314477473],
[-7.663364856925007,-9.269605292524087,-10.93661952691429,5.563297546729281,2.6219240462354607,0.22178301430844005,-3.4168642813984205],
[-6.3904143030522125,-10.374759695491537,-13.5353281119231,7.1068336458167956,3.439160466628407,0.025473590212108377,-3.4470424994759856],
[-4.7291392283448195,-11.15048811245629,-16.360041278607618,8.752034504089567,4.283459669117338,0.06159671498044528,-3.527500336643911],
[-2.687389059133894,-11.283371426297453,-19.10140510497603,10.442529878819828,5.0733249402586456,-0.23897304225410831,-3.5583460900263137],
[-0.31649578340475615,-10.649753662577817,-21.64918615897501,12.166059801709968,5.701406239170866,-0.9815325344663561,-3.520927660439506],
[2.3617628609598644,-9.115720434553026,-23.610316591246058,13.804328720993379,6.456748506417036,-1.9822723479531938,-3.5756699925465067],
[5.416285479284735,-6.863895543200452,-24.983626729868874,15.252069763640732,7.1956293367222655,-3.52462956916617,-3.574873420558313],
[8.81120567499087,-4.196928230919037,-25.743696194538163,16.432396867342888,7.762648477762421,-5.689042982722541,-3.4246725872248245],
[12.373521961954042,-1.3765981716497953,-25.914638448477564,17.283864706421383,7.956330935631105,-7.711081941402612,-3.2279280497923875],
[15.864621489382897,2.207254636085521,-25.714500551154998,17.727211111631185,7.49620283128164,-10.132784977860558,-2.753303252711181],
[18.947277056160672,5.466646615027308,-25.052900993553344,17.813320165002374,6.7418127631420095,-12.195758897198429,-2.223061794944195],
[21.305836427788797,8.665944454540336,-23.949768139392148,17.45155602395559,5.9081279822001,-13.9087082081939,-1.6222298973100626],
[22.906953543008438,11.764647620882666,-21.855088618811553,16.5429257646369,4.95790491922051,-15.61815250671549,-0.9536490933785817],
[23.79813983815978,14.234230230515031,-18.965340293601592,15.195102223698596,4.135146363882761,-17.206539727194187,-0.24835854378845212],
[24.07794434949703,15.831694455827162,-15.056814948504837,13.509864385240306,3.4194309607345232,-17.772396752682248,0.13224518819540165],
[23.83397797535696,17.238601385164372,-10.71939398596438,11.58487526352958,2.659916246911778,-18.168482296294467,0.4572345348497784],
[23.115170844587805,17.749163457540103,-6.5224067888004935,9.624975472858488,1.9420227245642825,-18.19866257239141,0.715151567950782],
[21.939194842772423,17.938738512858365,-3.03300827215478,7.709979057116055,1.337083186971812,-17.829502162273293,0.9092963348507599],
[20.37053606457971,17.33761329299547,0.2596225493391784,5.788069468379266,0.6833517478160029,-17.09381060045788,1.0003193608887102],
[18.626182291379436,16.11014162596456,2.9184258383686816,3.9783000813996217,0.10620265144406771,-15.712480877380036,0.8804337325662868],
[16.81565979270126,14.767469559918581,4.843734503064602,2.3780626763124335,-0.577482506705894,-14.107348762942106,0.6391786320698287],
[15.030010072252182,13.47815995052077,6.424992321840463,0.854325014096813,-1.3157770666927382,-12.261894848800388,0.24197928816638914],
[13.400150499819464,12.38521725091977,7.26380207050534,-0.4530742299654049,-2.3463516924019348,-10.77745578374019,-0.04836895039986899],
[11.947976564348998,11.296053395536344,7.468652844421359,-1.5220133652086316,-3.3959341760989865,-9.581682291783995,-0.3602777914404234],
[10.607692958343188,9.279561077728022,7.65703376629168,-2.546031432232989,-4.439033654236239,-8.147583955233939,-0.717731170741505],
[9.436503823490261,7.525171527521267,7.443912553184168,-3.4568080304384994,-5.353065140889107,-6.709588192919591,-1.0931408149624495],
[8.398026273070618,6.582392113802793,7.049332887100721,-4.308082637911629,-6.36281257255391,-5.978342619975142,-1.3019753305158828],
[7.485747330312935,6.033110145261684,6.563746422558134,-4.997390834539308,-7.27604470294504,-5.327694106785216,-1.5361069057712196]]

ref_cd = [[0.00538965,-0.00765677,-0.0452356,-0.030455,-0.0619395,-0.0838235,-0.0987388,-0.0739516,-0.0608395,-0.0395978,-0.0952817,-0.15024,-0.198386,-0.152795,-0.156822,-0.251269,-0.352601,-0.467646,-0.624065,-0.741789,-0.891667,-1.11646,-1.29588,-1.53039,-1.78431,-2.02833,-2.29662,-2.6368,-2.98907,-3.35758,-3.78046,-4.20962,-4.62796,-4.97665,-5.2335,-5.42042,-5.56606,-5.66497,-5.75632,-5.76988,-5.75085,-5.71927,-5.64417,-5.5679,-5.51888,-5.44424,-5.42402,-5.4639,-5.48608,-5.52795,-5.5905,-5.53719,-5.40617,-5.0572,-4.37624,-3.45504,-2.34642,-1.13284,0.205535,1.73085,3.44883,5.21041,7.04131,8.66812,10.0287,11.0595,11.7863,12.097,11.9016,11.4434,10.6104,9.52093,8.41149,7.23668,6.01864,5.073,4.27767,3.63388,3.21849,2.69014,2.19357],
[0.0414683,0.0209053,0.0291809,-0.0506169,-0.109469,-0.155978,-0.192383,-0.184673,-0.191489,-0.254137,-0.319511,-0.332366,-0.282746,-0.235686,-0.212261,-0.253465,-0.360791,-0.458675,-0.522105,-0.560396,-0.588016,-0.653722,-0.776516,-0.898141,-1.083,-1.27348,-1.41182,-1.53786,-1.79965,-2.00132,-2.21319,-2.48818,-2.7081,-2.87251,-3.1131,-3.2057,-3.23074,-3.31281,-3.34877,-3.38932,-3.43729,-3.46145,-3.51686,-3.57122,-3.60582,-3.58343,-3.53046,-3.48493,-3.48554,-3.62145,-3.74134,-3.77738,-3.74868,-3.58289,-3.29837,-2.96036,-2.47612,-1.84647,-1.17766,-0.357596,0.645737,1.57814,2.54232,3.37392,3.95596,4.39599,4.81352,5.14464,5.4214,5.44483,5.17109,4.76215,4.31846,3.84685,3.30232,2.74244,2.19205,1.76087,1.37635,0.904295,0.509128],
[0.0185152,0.0241566,0.0318504,0.043136,0.0585275,0.0887981,0.0862462,0.0877983,0.0960217,0.104761,0.106316,0.125327,0.114049,0.114066,0.118192,0.120264,0.134667,0.147019,0.152694,0.17122,0.188723,0.191322,0.205749,0.212979,0.220726,0.246995,0.297455,0.343298,0.373714,0.370167,0.337272,0.263177,0.17876,0.0695962,-0.0849315,-0.262174,-0.454412,-0.652891,-0.832358,-0.987134,-1.14299,-1.28754,-1.43162,-1.55151,-1.63273,-1.6825,-1.68379,-1.65875,-1.62131,-1.56631,-1.46637,-1.32612,-1.13159,-0.91841,-0.704652,-0.487728,-0.280562,-0.0965926,0.0145395,0.0440127,-0.0201028,-0.101822,-0.236843,-0.421568,-0.628597,-0.902962,-1.2302,-1.53735,-1.87099,-2.14104,-2.32779,-2.52599,-2.63411,-2.77594,-2.84994,-2.84206,-2.79324,-2.72574,-2.6017,-2.5606,-2.41835],
[-0.00289041,-0.0013021,-0.00125149,0.00287004,0.00826921,0.00411795,0.00256541,0.000231134,-0.0039784,-0.0188715,-0.0235418,-0.0336409,-0.0390365,-0.0401857,-0.0510804,-0.0796943,-0.0969232,-0.116076,-0.145532,-0.177696,-0.217743,-0.264729,-0.313644,-0.366027,-0.450739,-0.594869,-0.805167,-1.08009,-1.36695,-1.58147,-1.667,-1.66473,-1.61318,-1.58081,-1.58584,-1.61467,-1.65591,-1.71004,-1.78799,-1.87924,-1.99537,-2.09033,-2.18352,-2.27143,-2.35154,-2.42977,-2.5231,-2.62576,-2.74965,-2.90976,-3.07115,-3.25783,-3.47014,-3.68827,-3.92557,-4.11488,-4.28536,-4.36738,-4.35762,-4.28782,-4.0458,-3.73207,-3.2978,-2.77715,-2.28281,-1.73614,-1.13495,-0.621811,0.00716072,0.607412,1.05084,1.42137,1.60437,1.60668,1.59949,1.4716,1.21403,0.891872,0.495588,0.105151,-0.246813],
[8.53025e-05,0.00790959,0.0357791,0.00763463,0.0120934,-0.00750873,-0.0225689,-0.0214049,0.00858729,-0.000986512,0.01593,-0.00453073,-0.0293143,-0.0435578,-0.0347648,-0.0459442,-0.0351311,-0.0362666,-0.0568132,-0.0881031,-0.123247,-0.150259,-0.157379,-0.140999,-0.158656,-0.20323,-0.285209,-0.403832,-0.541119,-0.634821,-0.681599,-0.66394,-0.636395,-0.630437,-0.686076,-0.761396,-0.84953,-0.972358,-1.11957,-1.28739,-1.46758,-1.65551,-1.81393,-1.92502,-1.99665,-2.04904,-2.10302,-2.20786,-2.28038,-2.36977,-2.46311,-2.55935,-2.66457,-2.83196,-2.97865,-3.18524,-3.36189,-3.50604,-3.58876,-3.62962,-3.5768,-3.4194,-3.22103,-2.917,-2.58076,-2.20298,-1.84067,-1.33667,-0.847726,-0.237348,0.275366,0.725951,1.02325,1.28793,1.44737,1.48541,1.41228,1.23114,1.00076,0.742126,0.455299],
[-0.0335546,-0.00320802,-0.00230057,-0.00835194,-0.0223204,-0.00936638,0.0011042,0.0250137,0.0110427,0.0105977,0.0100901,0.00711977,-0.0123257,-0.0470557,-0.100578,-0.160721,-0.210025,-0.260124,-0.328559,-0.394505,-0.468129,-0.582066,-0.677524,-0.803348,-0.944271,-1.11263,-1.30574,-1.52429,-1.76432,-2.0295,-2.29953,-2.57054,-2.81911,-3.06603,-3.28458,-3.48139,-3.61709,-3.69473,-3.7671,-3.83741,-3.88877,-3.94355,-3.94441,-3.94899,-3.9402,-3.91411,-3.91016,-3.9272,-3.97133,-4.08485,-4.20146,-4.26719,-4.21941,-4.05846,-3.79172,-3.4415,-3.0561,-2.57192,-2.03706,-1.36633,-0.614193,0.259515,1.1739,2.05098,2.85691,3.58952,4.14148,4.56021,4.81478,4.9285,4.9506,4.71125,4.36303,3.8976,3.37325,2.88888,2.35485,1.76379,1.15532,0.572949,-0.000736615],
[-0.0186734,-0.000875444,0.00359564,0.00787171,-0.00481939,-0.00582518,0.00888932,0.0308508,0.047413,0.070429,0.0647037,0.0689022,0.0661231,0.0256538,0.0206948,0.0299215,0.0170853,5.89199e-05,0.0129955,-0.00452542,0.00451122,-0.0197851,-0.0543759,-0.114619,-0.150794,-0.179501,-0.199577,-0.230108,-0.325876,-0.446846,-0.62082,-0.820255,-1.07468,-1.29861,-1.5262,-1.73037,-1.9213,-2.09024,-2.31446,-2.47455,-2.58172,-2.68942,-2.75659,-2.74815,-2.74954,-2.74341,-2.71178,-2.71686,-2.68196,-2.60818,-2.53857,-2.46661,-2.36322,-2.20484,-1.9683,-1.65905,-1.26527,-0.75053,-0.200094,0.483136,1.21093,1.97978,2.692,3.31552,3.74841,3.9527,4.0366,4.00252,3.92276,3.72324,3.51627,3.22309,2.96456,2.67963,2.41631,2.08986,1.7493,1.43499,1.1229,0.800404,0.451836],
[0.0105745,-0.0128797,-0.0145573,-0.0028281,0.00890461,0.00722953,0.0223205,0.0156162,-0.0137317,-0.0321852,-0.0489666,-0.0749814,-0.087579,-0.0926268,-0.116142,-0.143022,-0.191738,-0.253065,-0.334566,-0.414411,-0.487559,-0.585097,-0.678459,-0.763447,-0.874514,-0.936014,-0.99501,-1.08514,-1.15259,-1.26885,-1.38093,-1.49052,-1.60856,-1.76873,-1.9222,-2.09424,-2.22591,-2.361,-2.46158,-2.58073,-2.70834,-2.81998,-2.90637,-2.95232,-2.94011,-2.88995,-2.82207,-2.75585,-2.69806,-2.6318,-2.5419,-2.39121,-2.15438,-1.82547,-1.38838,-0.826185,-0.162474,0.567317,1.29395,2.01234,2.67602,3.31791,3.83658,4.19561,4.38141,4.40997,4.32265,4.27084,4.22748,4.11133,3.9943,3.77904,3.49685,3.15276,2.7891,2.25593,1.69635,1.04342,0.35386,-0.365564,-0.978407],
[-0.0585159,-0.0507975,-0.0366412,-0.0113855,-0.00351417,0.0199423,0.0241033,0.00892747,-0.00901152,-0.00561238,-0.0345292,-0.029626,-0.0345544,-0.0545443,-0.0809901,-0.121298,-0.195464,-0.28237,-0.405269,-0.541666,-0.706602,-0.920564,-1.18366,-1.4824,-1.84985,-2.26662,-2.74041,-3.23918,-3.79598,-4.36476,-4.94447,-5.52765,-6.1027,-6.61845,-7.08334,-7.48502,-7.79071,-8.03236,-8.203,-8.30059,-8.32163,-8.28526,-8.18214,-8.06722,-7.94528,-7.84237,-7.78245,-7.78198,-7.83479,-7.95439,-8.10713,-8.20331,-8.11828,-7.73668,-6.97901,-5.84387,-4.329,-2.51558,-0.404493,2.021,4.72759,7.68007,10.7909,13.7047,16.2087,18.1295,19.3745,20.0449,20.2141,19.8867,19.0325,17.7557,16.1193,14.3245,12.519,10.8556,9.34548,8.07955,7.01317,6.15735,5.43743],
[-0.0363321,-0.0192361,0.109013,0.113296,0.0342051,-0.0555874,-0.0577297,-0.0769788,-0.0106923,-0.00427723,0.00427755,0.0470565,-0.0042782,-0.109102,-0.17971,-0.263167,-0.310262,-0.329543,-0.42373,-0.594979,-0.704181,-0.926849,-1.13885,-1.37657,-1.69996,-2.0962,-2.51177,-3.01306,-3.53157,-3.92379,-4.58166,-5.15395,-5.64276,-5.95599,-6.27781,-6.45185,-6.80376,-6.97785,-6.93766,-6.81171,-6.72434,-6.62837,-6.52596,-6.35276,-6.11953,-6.0578,-6.03036,-6.06728,-6.00766,-6.09177,-6.27885,-6.51744,-6.58444,-6.55489,-6.17128,-5.65885,-4.97896,-3.8504,-2.5199,-0.976694,0.860843,2.92193,5.09926,7.2941,9.05549,10.5229,11.5181,12.0731,12.2566,12.0319,11.3432,10.3622,9.16615,7.82812,6.53501,5.51681,4.56079,3.71854,3.09114,2.51522,2.06393],
[-0.0313357,-0.030162,-0.04484,-0.0606926,-0.0900496,-0.0941608,-0.100621,-0.0965125,-0.104147,-0.121176,-0.146426,-0.159934,-0.19047,-0.233339,-0.282081,-0.310272,-0.336114,-0.371353,-0.451811,-0.577489,-0.704348,-0.811242,-0.91579,-1.05264,-1.22297,-1.45909,-1.71107,-2.00417,-2.29611,-2.58981,-2.87354,-3.12615,-3.37465,-3.57911,-3.77417,-3.92636,-4.0263,-4.0604,-4.09163,-4.05701,-4.04826,-4.05595,-4.0472,-4.05196,-4.07319,-4.10438,-4.17436,-4.27369,-4.40359,-4.52999,-4.6411,-4.74459,-4.7846,-4.66714,-4.42106,-3.96222,-3.2842,-2.49159,-1.66606,-0.796427,0.0485518,0.898846,1.68687,2.40206,3.07378,3.73672,4.43844,5.06909,5.45466,5.64046,5.63762,5.38204,5.03478,4.52647,3.90825,3.30586,2.66231,1.98114,1.38575,0.77918,0.274866],
[-0.00290076,0.0288736,0.0249463,0.0106656,-0.0111118,-0.030033,-0.0318176,0.0092373,0.0213751,0.0142351,0.00174032,-0.0100402,-0.0368138,-0.0375276,-0.0493075,-0.0810779,-0.106065,-0.138906,-0.164606,-0.183167,-0.205297,-0.214933,-0.235635,-0.258834,-0.294527,-0.354493,-0.43837,-0.535098,-0.629683,-0.730689,-0.852755,-0.981243,-1.13329,-1.3096,-1.45129,-1.61046,-1.75749,-1.8749,-1.96376,-2.01157,-2.04831,-2.07471,-2.07005,-2.03542,-1.97259,-1.878,-1.76378,-1.63814,-1.50394,-1.33013,-1.15311,-0.932195,-0.674886,-0.359052,-0.00432697,0.35646,0.643014,0.852839,0.9267,0.968803,0.944886,0.926678,0.843527,0.679732,0.474903,0.326101,0.137335,0.0392064,-0.0628457,-0.200221,-0.35722,-0.467829,-0.62518,-0.762189,-0.930239,-1.11863,-1.40513,-1.72302,-2.07659,-2.3877,-2.69059],
[0.109542,0.0909708,0.103962,0.0116477,0.00797296,-0.00196123,-0.0488154,-0.122871,-0.0997191,-0.120915,-0.0597477,-0.0219871,-0.0439358,-0.167366,-0.227753,-0.29985,-0.36192,-0.384742,-0.394736,-0.570816,-0.727241,-0.91473,-1.22098,-1.45535,-1.79853,-2.16892,-2.52445,-2.85913,-3.30557,-3.73575,-4.31686,-4.84137,-5.28005,-5.65416,-6.0183,-6.30831,-6.52862,-6.6729,-6.79271,-6.85492,-6.85744,-6.7683,-6.60103,-6.53569,-6.54995,-6.5933,-6.672,-6.70192,-6.7332,-6.94716,-7.03959,-6.8975,-6.51995,-5.78394,-4.89968,-3.8041,-2.55016,-1.13928,0.32446,1.94329,3.63793,5.47428,7.31117,8.94077,10.4553,11.7808,13.0418,13.9503,14.3315,14.2547,13.4692,12.2945,11.1283,10.0583,8.96458,8.10524,7.00001,5.91917,4.87048,4.03971,3.48841],
[0.00214735,-0.011263,-0.0339012,-0.0485833,-0.0334968,-0.0385375,-0.0234423,-0.03226,-0.0331064,-0.0238776,-0.0352207,-0.0667294,-0.0684244,-0.0865078,-0.0924108,-0.0924311,-0.121464,-0.157659,-0.189243,-0.23178,-0.275598,-0.316911,-0.372136,-0.426119,-0.499081,-0.578817,-0.667016,-0.807518,-0.980122,-1.17894,-1.3981,-1.5933,-1.77129,-1.95231,-2.09542,-2.19552,-2.25048,-2.25393,-2.21006,-2.14251,-2.06097,-1.98744,-1.93332,-1.85338,-1.73237,-1.57237,-1.37591,-1.19248,-1.00347,-0.784302,-0.547254,-0.275355,0.0157334,0.313305,0.638566,0.976691,1.2946,1.6869,2.11247,2.6456,3.2282,3.82169,4.27782,4.58037,4.64042,4.45486,4.08516,3.56135,2.8834,2.24343,1.62955,1.0269,0.612126,0.186099,-0.237558,-0.571529,-0.970398,-1.35282,-1.64719,-1.99028,-2.30368],
[0.0344164,0.00218611,-0.0153094,-0.057982,-0.0722352,-0.0394181,-0.0071202,-0.0317808,-0.0586553,-0.0570353,-0.0384057,-0.0109778,-0.0406352,-0.0846016,-0.114866,-0.13361,-0.139718,-0.151334,-0.229026,-0.241792,-0.312426,-0.430527,-0.560312,-0.70014,-0.911289,-1.08672,-1.34849,-1.66519,-1.99765,-2.31213,-2.71155,-3.08253,-3.44273,-3.75782,-4.03269,-4.18359,-4.30966,-4.36423,-4.35164,-4.31461,-4.2892,-4.19817,-4.08148,-3.94912,-3.76598,-3.64172,-3.5474,-3.4904,-3.4785,-3.50563,-3.50377,-3.50134,-3.42965,-3.21484,-2.82766,-2.35235,-1.71101,-0.973416,-0.244157,0.496363,1.27057,2.09534,2.93875,3.68915,4.15725,4.31231,4.25961,4.16809,3.96854,3.76821,3.51654,3.2455,3.03731,2.95462,2.82393,2.7501,2.6384,2.44702,2.24362,2.06999,1.84704],
[-0.0125875,-0.00605167,0.00285345,0.00637372,0.00165185,0.00633466,0.00788482,-0.00825302,-0.0163366,-0.0305791,-0.050267,-0.0639242,-0.0691122,-0.097414,-0.108025,-0.10854,-0.129294,-0.157622,-0.178878,-0.221762,-0.237386,-0.271537,-0.320049,-0.394301,-0.499334,-0.606719,-0.690349,-0.799541,-0.89584,-1.01393,-1.17203,-1.32627,-1.48629,-1.65442,-1.80002,-1.94656,-2.09839,-2.26382,-2.43511,-2.60763,-2.73703,-2.85514,-2.93093,-2.98095,-2.98126,-2.94338,-2.83622,-2.70558,-2.57904,-2.48115,-2.41556,-2.39945,-2.35137,-2.27486,-2.18705,-2.0331,-1.81668,-1.51653,-1.05606,-0.509885,0.0901396,0.743175,1.44058,2.12573,2.78305,3.39691,3.83806,4.01834,4.01107,3.90243,3.70454,3.42281,3.0022,2.41628,1.7947,1.20193,0.569773,-0.102908,-0.639989,-1.15218,-1.45377],
[0.0750164,0.0747235,0.105484,0.128503,0.144667,0.11734,0.0805683,0.045974,0.0347037,0.0450624,0.0404834,0.0234509,0.00779346,0.0484136,0.079632,0.0909094,0.0864832,0.0852812,0.07206,0.0986247,0.141696,0.174476,0.197858,0.25019,0.271189,0.223637,0.181743,0.10117,-0.00733454,-0.0726452,-0.193122,-0.362926,-0.540636,-0.7333,-0.938676,-1.13161,-1.33642,-1.50882,-1.6255,-1.73693,-1.80519,-1.88496,-1.95549,-1.96316,-1.8888,-1.8435,-1.72372,-1.63232,-1.47984,-1.30991,-1.15842,-1.02788,-0.937604,-0.872847,-0.734187,-0.537041,-0.245485,0.260447,0.971806,1.81277,2.72274,3.67997,4.57756,5.25578,5.78461,6.11798,6.23258,6.1941,6.0425,5.67336,5.07248,4.44749,3.70275,2.94748,2.15629,1.42012,0.547594,-0.267388,-0.993486,-1.63894,-2.152],
[0.0289323,0.0302297,0.0175907,0.0199765,-0.013123,-0.00322399,0.00719902,-0.0140426,-0.049925,-0.0478218,-0.0731803,-0.0581858,-0.05931,-0.0373875,-0.0458659,-0.0774857,-0.111398,-0.104011,-0.0784997,-0.0204007,0.0106787,-0.0309169,-0.058613,-0.060002,-0.0304407,0.00427647,-0.0121535,-0.115539,-0.231242,-0.376644,-0.51665,-0.664076,-0.833452,-0.992107,-1.17639,-1.34356,-1.50136,-1.6975,-1.86337,-1.99164,-2.1195,-2.22138,-2.28579,-2.33462,-2.3281,-2.27006,-2.14584,-2.05147,-1.90794,-1.71282,-1.52363,-1.37972,-1.22885,-1.13301,-1.00425,-0.874259,-0.657232,-0.327748,0.211884,0.867593,1.56183,2.14965,2.63183,2.94957,3.14475,3.31616,3.5122,3.7472,3.93653,3.98878,3.9273,3.75186,3.49522,3.11126,2.58363,1.93471,1.34102,0.625805,-0.0969086,-0.823065,-1.47293],
[0.06287,0.046829,0.0386248,0.0103621,-0.0183012,-0.015539,0.0169697,0.0285959,0.0427825,0.0380758,0.0374179,0.0616914,0.0766709,0.0879063,0.105168,0.0642134,0.0204359,0.0345275,0.0572211,0.103,0.167749,0.165035,0.190246,0.236927,0.309186,0.402071,0.484229,0.490582,0.477032,0.410083,0.279209,0.157841,-3.79138e-05,-0.226339,-0.475769,-0.700219,-0.933686,-1.15506,-1.34637,-1.56007,-1.76848,-1.96056,-2.10208,-2.22407,-2.26192,-2.2313,-2.16248,-2.08506,-1.97854,-1.87145,-1.80793,-1.70939,-1.5751,-1.44797,-1.28479,-1.09579,-0.909221,-0.576448,-0.101944,0.522464,1.14759,1.6203,1.96275,2.21113,2.52984,2.91804,3.36729,3.83427,4.15977,4.40141,4.52839,4.39065,4.07524,3.58417,2.87005,2.16401,1.44728,0.799801,0.139729,-0.471203,-1.13717],
[-0.0366978,-0.00271219,0.0122923,-0.00842099,-0.00137478,-0.0170029,0.00232264,-0.0226647,-0.0367424,-0.0745172,-0.12432,-0.166516,-0.166635,-0.165657,-0.0992244,-0.0568141,-0.0521128,-0.0357943,-0.0894173,-0.119015,-0.099312,-0.0868899,-0.107704,-0.0924606,-0.16407,-0.259475,-0.333608,-0.447961,-0.574263,-0.686132,-0.847509,-1.0294,-1.22774,-1.44153,-1.67706,-1.98177,-2.30641,-2.60409,-2.8227,-2.99418,-3.10225,-3.20331,-3.29028,-3.29751,-3.21301,-3.13807,-2.95888,-2.76072,-2.5376,-2.33835,-2.14949,-2.01548,-1.93976,-1.94015,-1.96586,-2.06839,-2.05093,-1.91916,-1.59092,-1.0192,-0.244628,0.606897,1.45331,2.02366,2.19693,2.2784,2.38674,2.51444,2.93413,3.3551,3.50229,3.66447,3.68003,3.48776,3.35504,3.14127,2.73598,2.12366,1.34385,0.523055,-0.328981],
[0.128403,0.0564672,0.00510687,-0.0387131,-0.0690814,-0.0353728,0.032962,-0.0273834,-0.105939,-0.164284,-0.116596,-0.0570282,0.0021679,-0.0287238,-0.0736133,-0.0228188,0.0542908,0.0818687,0.0509448,0.0424811,0.0348588,0.0854063,0.104135,0.0956796,0.0757367,0.069814,0.00132985,-0.0395402,-0.141777,-0.266684,-0.438591,-0.620451,-0.770467,-0.938539,-1.15628,-1.34773,-1.56071,-1.78157,-1.93716,-2.10017,-2.19311,-2.25357,-2.18469,-2.10331,-1.994,-1.81895,-1.69397,-1.57532,-1.38827,-1.19333,-1.00983,-0.827893,-0.668177,-0.454086,-0.125695,0.302927,0.884922,1.60754,2.40325,3.07707,3.69789,4.12835,4.54967,4.84253,5.06512,5.08369,4.96505,4.70736,4.39699,3.84759,3.35536,2.77946,2.12107,1.53375,0.929863,0.290281,-0.264214,-0.776557,-1.24626,-1.63977,-2.03709],
[-0.0093371,-0.00472661,-0.0205913,-0.0419661,-0.046877,-0.0403989,-0.039414,-0.0342276,-0.0366721,-0.0480001,-0.0671814,-0.0795633,-0.103358,-0.132451,-0.173799,-0.226449,-0.306442,-0.380245,-0.475997,-0.60814,-0.773443,-0.965861,-1.19276,-1.43443,-1.7359,-2.0653,-2.42814,-2.81746,-3.22158,-3.64466,-4.065,-4.45342,-4.80968,-5.11604,-5.37946,-5.6077,-5.78058,-5.88777,-5.95249,-5.95275,-5.91524,-5.84243,-5.75744,-5.66325,-5.57191,-5.45593,-5.34247,-5.24515,-5.17922,-5.13962,-5.08989,-4.91257,-4.58557,-4.03914,-3.26638,-2.32653,-1.26704,-0.0864519,1.1854,2.57293,4.14709,5.77517,7.26184,8.48647,9.40934,9.93782,10.1791,10.1714,9.72811,9.1374,8.40202,7.55267,6.71778,5.87311,4.90816,4.04427,3.23391,2.49134,1.90033,1.3625,0.895341],
[-0.0239948,-0.0403132,-0.0448947,-0.0555751,-0.0356922,-0.0395381,-0.0385023,-0.0453078,-0.0431328,-0.0371881,-0.0298128,-0.0348626,-0.0473938,-0.0357011,-0.0230291,-0.0380347,-0.0571965,-0.109883,-0.198104,-0.273611,-0.374601,-0.488981,-0.617745,-0.771225,-0.958071,-1.1492,-1.37741,-1.62736,-1.91008,-2.22213,-2.57521,-2.93949,-3.30348,-3.64318,-3.95638,-4.23629,-4.46721,-4.64634,-4.7655,-4.82611,-4.87021,-4.87953,-4.85555,-4.78477,-4.70288,-4.61053,-4.54951,-4.49325,-4.45705,-4.41585,-4.40353,-4.38545,-4.3033,-4.07737,-3.6776,-3.08433,-2.36816,-1.54311,-0.636358,0.362633,1.46172,2.63087,3.83949,5.05029,6.07158,6.86234,7.34942,7.61923,7.69918,7.6727,7.55676,7.29388,6.92507,6.51015,5.98704,5.36344,4.76435,4.06663,3.43017,2.84345,2.32425],
[-0.0203914,-0.0215038,-0.0137364,0.00402428,-0.00319208,-0.00208205,-0.00707985,-0.00708073,-0.00930329,-0.0176368,-0.0448615,-0.0748708,-0.102109,-0.123241,-0.147157,-0.171635,-0.215022,-0.277326,-0.335752,-0.419221,-0.502708,-0.601242,-0.705916,-0.815626,-0.933711,-1.06518,-1.20949,-1.38946,-1.54554,-1.69553,-1.82829,-1.98615,-2.15073,-2.35938,-2.56751,-2.79966,-3.04747,-3.31373,-3.53267,-3.7517,-3.93393,-4.05767,-4.16025,-4.22044,-4.24441,-4.26223,-4.29011,-4.31353,-4.37711,-4.42847,-4.44242,-4.35921,-4.171,-3.88164,-3.48382,-2.94793,-2.30743,-1.57122,-0.776691,0.0308815,0.780518,1.49679,2.09471,2.69053,3.20877,3.57387,3.82715,3.90094,3.72234,3.48944,3.13782,2.62266,2.2059,1.73417,1.23153,0.792044,0.316045,-0.218892,-0.583643,-0.92271,-1.22207],
[-0.083169,-0.116883,-0.111643,-0.0958652,-0.0400406,-0.0137011,-0.0358416,-0.0601005,-0.109681,-0.130802,-0.131886,-0.138248,-0.147777,-0.140419,-0.174243,-0.193294,-0.196506,-0.220853,-0.211389,-0.232579,-0.283386,-0.315177,-0.30361,-0.322722,-0.330201,-0.367324,-0.420344,-0.413023,-0.326254,-0.261696,-0.245857,-0.295729,-0.411355,-0.52279,-0.596089,-0.666238,-0.690787,-0.748255,-0.837589,-0.967313,-1.16082,-1.28855,-1.37701,-1.48253,-1.60084,-1.69475,-1.8238,-1.9412,-2.08206,-2.25809,-2.44911,-2.61785,-2.73129,-2.87993,-2.95407,-3.00159,-3.08537,-3.15533,-3.24025,-3.43718,-3.57447,-3.6798,-3.71155,-3.49141,-3.20498,-2.86396,-2.416,-1.96142,-1.52908,-1.05488,-0.630714,-0.206363,0.194644,0.50276,0.711509,0.806908,0.779252,0.648802,0.403715,0.160664,-0.0707075],
[-0.125323,-0.0968837,-0.107615,-0.0951474,-0.072991,-0.0652577,-0.0512992,-0.0331752,-0.0729836,-0.0483202,-0.0162542,-0.0360578,-0.0786287,-0.133696,-0.203937,-0.285288,-0.291602,-0.346186,-0.390348,-0.436964,-0.484398,-0.530875,-0.530492,-0.585415,-0.650399,-0.768759,-0.891661,-1.05284,-1.22708,-1.39551,-1.59352,-1.86016,-2.18319,-2.56476,-3.02491,-3.40865,-3.73599,-4.04903,-4.31771,-4.52116,-4.77005,-4.89053,-5.00231,-5.09979,-5.08013,-5.0382,-4.97087,-4.87961,-4.90642,-4.98666,-5.09902,-5.20845,-5.19366,-4.97224,-4.62939,-3.99796,-3.24203,-2.34977,-1.42733,-0.398207,0.545614,1.5139,2.51611,3.44369,4.35334,5.16754,5.78318,6.09619,6.26992,6.17552,5.99043,5.73547,5.34254,4.85034,4.37823,3.82398,3.19483,2.44715,1.6486,0.894798,0.383326],
[0.0800398,0.0629708,0.0577404,0.047898,0.0270087,0.0352072,0.0508699,0.0489132,0.0849863,0.0871137,0.0485521,0.00611296,0.048237,0.0291325,0.000761003,-0.0833797,-0.175148,-0.290467,-0.387603,-0.577205,-0.738839,-0.95087,-1.13521,-1.3077,-1.53589,-1.83719,-2.13182,-2.47149,-2.76188,-3.06303,-3.39731,-3.78392,-4.15604,-4.5471,-4.90916,-5.25066,-5.49983,-5.72994,-5.83494,-5.88791,-5.88654,-5.8107,-5.82339,-5.83907,-5.84871,-5.79628,-5.80283,-5.81777,-6.01598,-6.12775,-6.26292,-6.3132,-6.24787,-5.94224,-5.43665,-4.68232,-3.84331,-2.90286,-1.76015,-0.423279,1.12117,2.76199,4.31317,5.70826,6.85282,7.71047,8.33672,8.71946,8.84953,8.70599,8.34181,7.81926,7.1367,6.36021,5.64255,4.88217,4.24538,3.63216,2.99266,2.35611,1.79531],
[0.0988193,0.0637285,0.136845,0.0875941,0.178781,0.209579,0.225936,0.234286,0.216198,0.12273,0.113682,0.0792566,0.0186707,-0.0351538,-0.0355289,-0.12048,-0.189624,-0.151969,-0.284046,-0.364005,-0.424795,-0.698017,-1.00134,-1.23137,-1.47413,-1.64119,-1.87347,-2.20804,-2.6356,-3.007,-3.36983,-3.63333,-3.86491,-4.1044,-4.28169,-4.38158,-4.47673,-4.43551,-4.32782,-4.2416,-4.11393,-4.05041,-4.03413,-3.91727,-3.77649,-3.64989,-3.51502,-3.36182,-3.37213,-3.32499,-3.27672,-3.31777,-3.23951,-2.85798,-2.54579,-2.13518,-1.32756,-0.424872,0.824965,2.32437,3.78598,5.17499,6.52747,7.51412,8.26031,8.53742,8.52624,8.27309,7.95068,7.45166,7.0428,6.15624,5.2129,4.52213,3.96879,3.6199,3.26877,2.42351,1.67078,1.34281,1.00738],
[-0.00793281,0.00158391,0.0316834,0.0311347,0.0290048,0.0163379,0.0152733,-0.000525636,-0.019458,-0.0278536,-0.00892779,-0.00629764,-0.0131121,-0.0424534,-0.0717534,-0.0942093,-0.125002,-0.183977,-0.282045,-0.40242,-0.551838,-0.720854,-0.920367,-1.1576,-1.46264,-1.81976,-2.25014,-2.74948,-3.30257,-3.88445,-4.48419,-5.05769,-5.59007,-6.09481,-6.55029,-6.91634,-7.22924,-7.44122,-7.56479,-7.60996,-7.58661,-7.48211,-7.37053,-7.2201,-7.06422,-6.89218,-6.73212,-6.59636,-6.53546,-6.57072,-6.64525,-6.71459,-6.6721,-6.39113,-5.79423,-4.86382,-3.60865,-2.10521,-0.338769,1.69873,4.04068,6.62666,9.27224,11.6423,13.4435,14.4953,14.8465,14.6346,14.029,13.0706,11.8925,10.565,9.18067,7.87665,6.72561,5.72827,4.94949,4.32963,3.86037,3.49185,3.21933],
[-0.0484694,-0.0223631,-0.00319066,-0.0163895,-0.0567443,-0.0653017,-0.0566511,-0.0990929,-0.133622,-0.155001,-0.15371,-0.179256,-0.204668,-0.226356,-0.243956,-0.26074,-0.310959,-0.468576,-0.618469,-0.801861,-1.02226,-1.26767,-1.58468,-1.99736,-2.35607,-2.79344,-3.31899,-3.83764,-4.40266,-5.02311,-5.57633,-6.16229,-6.77218,-7.26133,-7.72007,-8.11225,-8.32863,-8.537,-8.66856,-8.68563,-8.67435,-8.61892,-8.46832,-8.40694,-8.41738,-8.41809,-8.44842,-8.51133,-8.58117,-8.77609,-8.9548,-9.01986,-8.89575,-8.50991,-7.76881,-6.78998,-5.42389,-3.78076,-1.91979,0.202416,2.58966,5.12017,7.72178,10.1178,12.1491,13.7336,14.8449,15.5985,15.9425,15.9035,15.625,14.9479,14.0483,13.1545,12.0995,11.0346,9.99296,8.87793,7.88529,7.0698,6.30692],
[-0.00674945,-0.00727576,-0.00669282,-0.00867572,0.000262838,0.00158246,0.00253948,0.0102872,0.00747664,-0.00131255,-0.00690818,-0.00954615,-0.00501961,-0.00443807,-0.00623657,-0.0134786,-0.0196385,-0.0145811,-0.0139483,-0.0156186,-0.0187572,-0.0199611,-0.0187298,-0.00908882,-0.00490865,-0.0119249,-0.0319209,-0.055738,-0.0823159,-0.101442,-0.114224,-0.131035,-0.149613,-0.176692,-0.209747,-0.25548,-0.303016,-0.348238,-0.403605,-0.450536,-0.499943,-0.542198,-0.569401,-0.572256,-0.566626,-0.540914,-0.514523,-0.477199,-0.425729,-0.35573,-0.271283,-0.186019,-0.0745717,0.0405905,0.18499,0.321474,0.442531,0.514822,0.549542,0.518078,0.46825,0.384887,0.323588,0.261522,0.191497,0.122462,0.0622557,-0.0159841,-0.0719515,-0.160176,-0.275922,-0.383407,-0.524345,-0.679207,-0.83051,-1.01809,-1.19018,-1.34107,-1.49296,-1.62126,-1.72079],
[0.00178583,-0.00582795,0.0109263,-0.000933946,-0.0210693,-0.0311415,-0.0101807,-0.0102667,-0.000507879,-0.0181416,-0.0349421,-0.0463258,-0.0355847,-0.0407506,-0.0590343,-0.102684,-0.135157,-0.179485,-0.224236,-0.302078,-0.401532,-0.543015,-0.630683,-0.730149,-0.826803,-0.93477,-1.03391,-1.15592,-1.22665,-1.34252,-1.49792,-1.67773,-1.84679,-2.04456,-2.24226,-2.43205,-2.64736,-2.85065,-3.02036,-3.21544,-3.39147,-3.52842,-3.64892,-3.73458,-3.78754,-3.85876,-3.91739,-3.92717,-3.91343,-3.92746,-3.99426,-4.114,-4.1487,-3.99195,-3.64724,-3.10678,-2.43258,-1.66258,-0.722846,0.259062,1.20748,2.12246,2.87397,3.43548,3.83188,3.89986,3.97165,4.07115,4.22161,4.26297,4.19297,3.95884,3.83579,3.5801,3.431,3.08077,2.5988,2.19339,1.86844,1.38208,1.01915],
[0.0279049,0.0131467,0.0127107,0.00822134,0.00681847,0.00592938,0.0111665,0.00497391,0.013149,0.0106403,0.0047437,-0.0112873,-0.0203916,-0.0387855,-0.0398197,-0.0453618,-0.0381116,-0.0382797,-0.0671112,-0.105409,-0.122053,-0.153278,-0.208632,-0.287925,-0.39526,-0.532448,-0.668937,-0.825992,-0.980902,-1.10147,-1.18919,-1.25437,-1.30466,-1.33021,-1.33255,-1.31754,-1.31022,-1.30018,-1.2928,-1.25487,-1.20879,-1.16692,-1.11406,-1.0518,-0.981609,-0.88366,-0.72896,-0.567705,-0.330224,-0.0607699,0.211757,0.515216,0.858339,1.20611,1.61559,2.09448,2.52713,2.956,3.37465,3.7543,4.08167,4.46722,4.83629,5.15847,5.37518,5.42901,5.22332,4.75657,4.18618,3.42247,2.42269,1.43994,0.420902,-0.568786,-1.38102,-2.07552,-2.66182,-2.9943,-3.31553,-3.61146,-3.86727],
[-0.237618,0.128904,0.217923,0.18835,0.189994,-0.106857,-0.0709426,-0.0490751,-0.258782,-0.258894,-0.0648003,-0.0491597,0.0903283,0.112321,-0.140217,-0.26898,-0.1529,-0.255073,-0.431198,-0.351199,-0.604588,-0.979361,-1.14351,-1.70782,-2.11191,-2.31142,-2.69877,-3.16059,-3.62912,-4.23384,-4.7522,-5.22991,-5.73964,-6.15648,-6.50723,-6.7427,-6.73291,-6.75004,-6.79729,-6.77798,-6.66188,-6.65837,-6.32936,-6.30186,-6.33633,-6.38988,-6.47212,-6.76763,-6.8485,-7.02497,-7.25099,-7.36881,-7.44687,-7.47873,-7.09889,-6.52874,-5.60992,-4.35956,-2.92592,-1.13284,1.01053,3.24053,5.63414,7.77666,9.51553,11.0154,12.1652,12.8619,13.1739,13.1811,12.8251,12.059,11.1234,10.1178,9.02449,8.17337,7.40366,6.47057,5.79784,5.2455,4.64913],
[0.0523552,0.079523,0.0841733,0.0631911,0.0402323,0.0257896,0.0102011,-0.0121693,-0.0274217,-0.0345536,-0.0400735,-0.0397267,-0.0609228,-0.0742917,-0.0991041,-0.119494,-0.146516,-0.167327,-0.221626,-0.286508,-0.373525,-0.500016,-0.648726,-0.818899,-1.00825,-1.24831,-1.51906,-1.82288,-2.13617,-2.41857,-2.62399,-2.77816,-2.89527,-3.00936,-3.12179,-3.23104,-3.336,-3.4111,-3.47366,-3.52102,-3.56655,-3.59524,-3.63679,-3.65969,-3.67847,-3.69794,-3.76218,-3.83885,-3.94805,-4.09971,-4.29575,-4.49231,-4.64418,-4.67871,-4.54505,-4.23467,-3.74956,-3.12011,-2.31224,-1.33318,-0.217735,1.00716,2.25217,3.35289,4.27392,4.99968,5.47521,5.80926,5.99826,6.00522,5.89491,5.63131,5.19523,4.66164,4.07288,3.472,2.93923,2.40459,1.9377,1.4648,1.0807],
[-0.076537,-0.07426,-0.0918656,-0.00315528,-0.00774647,0.0542373,0.0519529,0.039715,-0.0452754,-0.0667315,-0.100454,-0.0912808,-0.16028,-0.156482,-0.203289,-0.225575,-0.301567,-0.341532,-0.528093,-0.716273,-0.889179,-1.14587,-1.47871,-1.7318,-2.10257,-2.49887,-2.92532,-3.50806,-4.15028,-4.82662,-5.46095,-6.03244,-6.51179,-6.97442,-7.34638,-7.66687,-7.8912,-8.03551,-8.07427,-8.07831,-7.98063,-7.81885,-7.68787,-7.53598,-7.41567,-7.29687,-7.23668,-7.22589,-7.30626,-7.40676,-7.49723,-7.51279,-7.3397,-6.86258,-6.07124,-4.91438,-3.43745,-1.77333,0.177188,2.28884,4.65002,7.19507,9.71641,12.0776,14.012,15.3843,16.2794,16.6902,16.7458,16.429,15.7075,14.6761,13.4423,12.1559,10.893,9.60005,8.41771,7.36076,6.4705,5.77043,5.14936],
[0.0429076,0.0528482,0.0600337,0.0473292,0.0181873,0.0116496,0.00727446,0.0246376,0.0509624,0.0465045,0.0116395,-0.0362871,-0.0384502,-0.0925609,-0.133153,-0.210834,-0.30464,-0.401674,-0.491512,-0.656444,-0.871816,-1.16197,-1.52471,-1.89511,-2.32059,-2.82464,-3.36058,-3.95225,-4.62625,-5.3372,-6.01971,-6.6505,-7.18711,-7.63925,-8.05441,-8.37717,-8.5782,-8.66252,-8.66047,-8.56715,-8.45594,-8.27451,-8.07312,-7.87924,-7.71022,-7.55496,-7.47625,-7.44311,-7.48221,-7.63236,-7.76703,-7.75921,-7.4942,-6.88063,-5.88797,-4.6317,-3.07243,-1.22093,0.895979,3.22777,5.78507,8.4386,11.0321,13.3731,15.2064,16.4258,17.1135,17.3342,17.1671,16.6555,15.8042,14.662,13.4105,12.0643,10.7238,9.44284,8.29001,7.28226,6.43617,5.75064,5.15805],
[-0.0544626,-0.0385471,-0.0450171,-0.040641,-0.02943,-0.0277071,-0.0205586,-0.0243342,-0.0644018,-0.0954469,-0.119837,-0.130772,-0.13991,-0.155296,-0.184161,-0.218242,-0.270647,-0.316042,-0.377707,-0.462077,-0.574549,-0.708409,-0.872673,-1.06919,-1.26392,-1.49752,-1.74079,-2.00807,-2.28877,-2.6112,-2.92339,-3.2491,-3.53292,-3.78341,-3.96695,-4.10365,-4.19115,-4.21061,-4.15072,-4.04209,-3.8731,-3.68604,-3.52329,-3.37517,-3.25055,-3.13596,-3.00965,-2.92409,-2.88821,-2.90162,-2.93455,-2.87533,-2.69233,-2.39393,-1.89522,-1.29075,-0.59116,0.179432,1.00837,1.91388,2.85352,3.73744,4.34231,4.78517,4.9587,5.01297,4.90299,4.75783,4.40279,4.00035,3.54537,3.04084,2.49686,1.96849,1.34172,0.660181,0.147252,-0.318758,-0.655785,-0.913179,-1.19002],
[-0.0503988,-0.0858804,-0.104009,-0.116556,-0.0630133,-0.067388,-0.0734696,-0.0512126,-0.0583139,-0.0939751,-0.086784,-0.116277,-0.135646,-0.126929,-0.130084,-0.181147,-0.211644,-0.231044,-0.294517,-0.364928,-0.489052,-0.624059,-0.770057,-0.924826,-1.09871,-1.2567,-1.48723,-1.70468,-2.00413,-2.34588,-2.68864,-2.99053,-3.26547,-3.49741,-3.79107,-4.04144,-4.27072,-4.46124,-4.58348,-4.65837,-4.75324,-4.73569,-4.68845,-4.60366,-4.45329,-4.30291,-4.18957,-4.11036,-4.02948,-4.0278,-4.01535,-4.00403,-3.93943,-3.76418,-3.39201,-2.8953,-2.29084,-1.64081,-0.959288,-0.144824,0.709724,1.76274,2.86805,3.93011,4.99343,5.89881,6.54785,7.03452,7.26509,7.32184,7.37057,7.20722,6.94467,6.50039,5.8602,5.24006,4.5653,3.86036,3.04832,2.23524,1.42289],
[-0.0493659,-0.0370536,-0.0507352,-0.0896414,-0.115397,-0.0848818,0.0143432,0.0453786,0.0500484,0.0460992,-0.0207005,-0.081845,-0.0691718,-0.108771,-0.177857,-0.22981,-0.255916,-0.319772,-0.362896,-0.418893,-0.538876,-0.613542,-0.700505,-0.931354,-1.08701,-1.21099,-1.35601,-1.48753,-1.70425,-2.06459,-2.3332,-2.60098,-2.85804,-3.10159,-3.27203,-3.54296,-3.77348,-3.98463,-4.11518,-4.23293,-4.27256,-4.26413,-4.27349,-4.29123,-4.35569,-4.37103,-4.30363,-4.18475,-4.19424,-4.16548,-4.2169,-4.2292,-4.08965,-3.82446,-3.56367,-3.00937,-2.30026,-1.59392,-0.72672,0.210903,1.16663,2.33753,3.62322,4.80005,5.74179,6.21753,6.2382,6.12417,6.06193,6.00334,5.8883,5.56512,4.95441,4.29339,3.72565,3.16476,2.5998,2.02189,1.4921,1.07131,0.627357],
[0.016694,0.0033393,-0.0400775,-0.0300626,-0.0459358,-0.0534604,-0.0643292,-0.0793793,-0.0777195,-0.0668654,-0.0727269,-0.0961479,-0.139644,-0.159737,-0.176489,-0.194919,-0.208336,-0.221756,-0.26782,-0.322269,-0.388455,-0.457172,-0.528423,-0.612257,-0.699468,-0.797593,-0.939325,-1.12635,-1.34193,-1.61877,-1.86634,-2.07878,-2.27953,-2.43588,-2.58138,-2.68664,-2.77767,-2.84523,-2.8935,-2.93003,-2.94726,-2.92838,-2.90697,-2.89985,-2.89775,-2.91751,-2.91206,-2.94274,-2.99278,-3.08235,-3.17363,-3.22372,-3.16783,-3.05305,-2.82126,-2.51452,-2.19843,-1.90328,-1.69898,-1.57629,-1.44599,-1.26427,-0.966242,-0.563653,-0.0792097,0.519997,1.11685,1.68101,2.26219,2.73729,3.1012,3.37918,3.40836,3.32364,3.17729,2.98617,2.7781,2.61134,2.35081,2.14847,1.95452],
[-0.0232311,-0.0143265,-0.00860909,-0.0114315,-0.0250882,-0.0417472,-0.0449307,-0.0296403,-0.0263634,-0.0313641,-0.0295408,-0.0298517,-0.0438832,-0.0464298,-0.0660904,-0.0919768,-0.110606,-0.122208,-0.144346,-0.171326,-0.209111,-0.276791,-0.36128,-0.457523,-0.590996,-0.743133,-0.924634,-1.14702,-1.32796,-1.4557,-1.53039,-1.54142,-1.54496,-1.57333,-1.58217,-1.56321,-1.5254,-1.48037,-1.43154,-1.41533,-1.41449,-1.40948,-1.37855,-1.33427,-1.23929,-1.11023,-0.931599,-0.726367,-0.490135,-0.223975,0.0780753,0.409549,0.810716,1.25949,1.74046,2.24165,2.7439,3.20766,3.60377,3.90867,4.13427,4.3809,4.64555,4.9069,5.05325,4.94866,4.68619,4.34852,3.887,3.31493,2.6232,1.79674,0.924491,0.154518,-0.591282,-1.28048,-1.8049,-2.26788,-2.71265,-3.09146,-3.52602],
[0.00810985,0.00242552,0.00264193,-0.0100627,-0.0341976,-0.0307243,-0.0332063,-0.0425908,-0.0412737,-0.0467857,-0.0659588,-0.0858547,-0.115342,-0.149189,-0.177287,-0.21052,-0.257337,-0.321772,-0.397441,-0.492833,-0.601289,-0.716626,-0.839897,-0.980102,-1.13316,-1.30902,-1.51891,-1.75712,-2.01277,-2.27544,-2.50722,-2.71096,-2.88943,-3.03915,-3.16905,-3.27965,-3.38189,-3.48614,-3.57849,-3.66495,-3.73696,-3.79127,-3.82723,-3.83466,-3.79442,-3.71886,-3.61751,-3.50301,-3.37208,-3.22003,-3.02992,-2.83038,-2.62895,-2.41623,-2.19602,-1.92431,-1.58734,-1.18973,-0.708329,-0.135485,0.513381,1.19525,1.91976,2.5992,3.22095,3.7234,4.1517,4.29815,4.3283,4.17436,3.81919,3.30564,2.73137,1.98535,1.26539,0.52118,-0.206143,-0.863363,-1.44035,-1.96737,-2.39127],
[0.0124548,0.0236607,0.0269341,0.0365135,0.0218078,0.0143374,0.0197146,0.0278986,0.0272034,0.0370294,0.0316587,0.0410221,0.0305027,0.0262973,0.0169404,0.0122618,-0.00529804,0.0071144,0.0214065,0.0232857,0.0162596,0.0087616,-0.00999451,-0.0334494,-0.0374445,-0.0536422,-0.0684388,-0.084885,-0.103686,-0.141752,-0.198861,-0.288415,-0.414902,-0.572469,-0.736453,-0.908268,-1.07287,-1.23072,-1.38816,-1.53391,-1.66113,-1.76038,-1.82741,-1.8688,-1.90243,-1.93583,-1.96454,-1.97251,-1.96421,-1.92737,-1.88226,-1.81896,-1.71362,-1.56645,-1.35948,-1.11534,-0.82952,-0.504352,-0.19795,0.115203,0.425893,0.708576,0.991382,1.23078,1.37705,1.48244,1.56491,1.58631,1.48908,1.27573,0.956614,0.527394,0.107706,-0.320462,-0.770396,-1.12541,-1.42933,-1.69897,-1.92222,-2.137,-2.29038],
[0.00161619,0.0140539,0.00381843,-0.0156573,-0.0280634,-0.0588119,-0.0563941,-0.0405179,-0.03743,-0.0488484,-0.0454726,-0.0712459,-0.0634128,-0.0667425,-0.0894588,-0.149753,-0.204735,-0.290672,-0.372142,-0.457452,-0.552505,-0.69495,-0.835093,-1.01766,-1.23975,-1.45587,-1.71349,-2.00583,-2.32801,-2.65456,-2.96862,-3.23352,-3.48532,-3.72299,-3.93064,-4.10534,-4.23644,-4.34668,-4.44378,-4.48925,-4.5122,-4.52086,-4.53488,-4.52349,-4.51994,-4.48243,-4.46327,-4.4814,-4.56242,-4.63117,-4.69346,-4.71718,-4.60606,-4.37118,-4.03309,-3.54906,-2.90186,-2.15648,-1.26409,-0.231841,0.893828,2.16807,3.45697,4.72905,5.81708,6.63343,7.23966,7.65833,7.73475,7.65309,7.32946,6.8245,6.28006,5.67458,4.94232,4.2452,3.47553,2.70361,2.00671,1.32424,0.715863],
[-0.034982,-0.0238706,-0.0123495,-0.00603626,-0.0330182,-0.0407052,-0.0527869,-0.0500586,-0.0573101,-0.0605514,-0.100401,-0.102085,-0.132052,-0.126488,-0.163805,-0.214037,-0.307052,-0.35119,-0.427521,-0.479814,-0.567431,-0.719661,-0.897216,-1.0595,-1.24355,-1.45845,-1.68604,-1.97931,-2.27594,-2.61783,-2.99041,-3.33933,-3.64939,-3.93748,-4.13457,-4.36282,-4.57809,-4.72636,-4.84141,-4.90805,-4.92158,-4.94084,-4.91773,-4.86981,-4.83771,-4.80214,-4.77631,-4.78096,-4.7105,-4.70834,-4.69762,-4.66627,-4.54848,-4.24135,-3.66997,-2.99686,-2.15902,-1.27372,-0.435089,0.517971,1.59865,2.74147,3.99307,5.1616,6.18062,6.95686,7.48588,7.62544,7.69352,7.5024,7.42272,7.22855,6.75933,6.14029,5.40238,4.45729,3.72298,3.02707,2.29826,1.81951,1.2597],
[-0.0270496,0.026237,0.0432339,0.0473674,-0.0224653,-0.00784789,0.00785564,0.135401,0.14965,0.0558825,-0.0211599,-0.0288834,0.0289883,0.0726035,0.036991,-0.070644,-0.199509,-0.227418,-0.231344,-0.336828,-0.533493,-0.789883,-1.04371,-1.29814,-1.61828,-1.93858,-2.3104,-2.66462,-2.9514,-3.326,-3.66983,-3.9643,-4.23814,-4.46809,-4.61867,-4.71919,-4.7296,-4.71337,-4.72425,-4.70654,-4.70516,-4.64439,-4.54805,-4.41392,-4.34341,-4.31815,-4.39392,-4.45748,-4.51855,-4.59516,-4.61258,-4.62524,-4.55068,-4.28697,-3.9346,-3.46772,-2.81242,-1.98602,-1.11201,-0.0954855,0.979101,2.12507,3.27991,4.38281,5.26754,5.99496,6.52022,6.83356,6.95974,6.80725,6.47138,6.00709,5.57638,5.06374,4.63667,4.1932,3.76426,3.37319,3.05762,2.73535,2.44443],
[0.0636864,-0.024773,-0.0511094,-0.0652818,-0.0351175,0.0555724,0.0820151,0.0530818,0.0821433,0.0441291,-0.024948,-0.0598732,-0.0631544,-0.0593191,-0.0464068,-0.0795122,-0.141221,-0.217313,-0.296123,-0.384811,-0.471683,-0.612758,-0.772305,-0.948408,-1.1287,-1.36546,-1.57382,-1.8126,-2.06812,-2.31092,-2.47668,-2.61644,-2.70582,-2.82361,-2.91131,-2.97412,-2.98825,-3.01362,-3.07861,-3.13645,-3.21089,-3.2378,-3.21246,-3.21026,-3.2253,-3.23306,-3.32383,-3.40476,-3.43594,-3.41325,-3.42182,-3.45973,-3.39164,-3.30475,-2.97389,-2.44797,-1.87108,-1.13263,-0.222878,0.671482,1.71079,2.77516,3.73898,4.64168,5.38015,5.92843,6.28136,6.45402,6.48516,6.31096,5.97829,5.48071,4.82325,4.19667,3.62271,3.03705,2.49736,1.93717,1.36186,1.03004,0.688195],
[-0.0217149,0.000245948,-0.0333265,-0.100112,-0.16926,-0.164051,-0.147202,-0.0801523,-0.032961,-0.0664886,-0.125452,-0.136638,-0.133427,-0.0620226,-0.0299488,-0.0660033,-0.0983073,-0.230874,-0.307368,-0.441358,-0.537583,-0.651343,-0.836416,-1.07888,-1.2393,-1.48662,-1.67128,-1.92193,-2.30867,-2.69414,-3.03627,-3.30845,-3.56889,-3.84102,-4.03387,-4.24295,-4.37867,-4.47855,-4.58521,-4.68017,-4.70312,-4.78094,-4.86556,-5.01678,-5.00946,-4.95436,-4.8342,-4.7848,-4.74899,-4.85876,-4.75933,-4.74943,-4.65243,-4.42066,-3.94745,-3.26523,-2.46272,-1.54209,-0.545418,0.635008,1.94581,3.3147,4.49133,5.54348,6.432,7.00071,7.28781,7.2013,7.01859,6.778,6.48127,5.95022,5.3432,4.45652,3.77404,3.20941,2.49654,1.75435,1.20073,0.556068,0.122388],
[0.00219032,0.00612042,0.00723177,-0.0157555,0.000680926,-0.0476604,0.0219852,0.0334087,0.0283706,0.008669,-0.025816,-0.103839,-0.125243,-0.0967386,-0.0803673,-0.0770571,-0.112276,-0.133425,-0.187575,-0.230197,-0.327605,-0.435258,-0.553373,-0.639042,-0.77444,-0.903436,-1.05128,-1.1907,-1.33288,-1.44551,-1.61445,-1.77393,-1.9406,-2.12273,-2.28559,-2.41548,-2.51387,-2.5924,-2.64068,-2.71512,-2.79752,-2.89884,-2.95319,-3.01403,-3.04272,-3.08116,-3.10652,-3.16907,-3.21349,-3.24136,-3.2906,-3.28662,-3.16548,-2.95359,-2.59435,-2.11232,-1.59399,-1.05614,-0.379912,0.406355,1.32739,2.33137,3.37529,4.28891,5.00924,5.46661,5.73614,5.8259,5.73816,5.49799,5.14548,4.70847,4.22313,3.68803,3.0605,2.3601,1.69914,1.08394,0.504654,-0.0712608,-0.592856],
[0.0368082,0.0352115,0.0140622,0.0162544,0.0282279,0.0232699,0.0124681,0.00677407,0.0184639,0.0324969,0.038643,0.0269555,0.033542,0.0231645,0.0477474,0.0411732,0.0515738,0.055684,0.0574534,0.0400375,0.0497145,0.03522,0.0248225,0.00254692,-0.0486218,-0.160233,-0.253556,-0.373184,-0.524707,-0.684221,-0.861424,-1.07718,-1.29625,-1.50617,-1.7222,-1.93656,-2.16366,-2.37101,-2.57742,-2.74539,-2.86074,-2.94217,-3.02803,-3.10937,-3.17911,-3.2421,-3.27567,-3.31101,-3.36081,-3.43318,-3.49498,-3.52966,-3.51464,-3.4319,-3.26705,-3.02289,-2.67628,-2.26101,-1.78381,-1.23505,-0.657088,-0.0385214,0.616682,1.15476,1.59723,1.93338,2.0985,2.17724,2.15921,2.03427,1.85966,1.68276,1.51481,1.40045,1.32471,1.25161,1.14052,1.03843,0.935402,0.808588,0.652632],
[0.0286264,0.0445477,0.0488039,0.0550422,0.0664454,0.0560119,0.0573819,0.0547455,0.0635575,0.0857032,0.111011,0.103654,0.0928825,0.065843,0.0692187,0.063869,0.0622632,0.0517813,0.0394959,0.0512665,0.0784751,0.0830171,0.0985309,0.111404,0.115283,0.135383,0.140455,0.131454,0.103184,0.0876923,0.0936522,0.101849,0.0958152,0.0750314,0.0187287,-0.0173439,-0.0533569,-0.101714,-0.16843,-0.249203,-0.369437,-0.513462,-0.691458,-0.883085,-1.09344,-1.31569,-1.56606,-1.85645,-2.1782,-2.53044,-2.9193,-3.33016,-3.7728,-4.22578,-4.68595,-5.15436,-5.58895,-5.95105,-6.20856,-6.28798,-6.12621,-5.71454,-5.07041,-4.26313,-3.33043,-2.34048,-1.30878,-0.261431,0.624036,1.28842,1.74244,1.98412,2.02059,1.93614,1.7003,1.34137,0.924187,0.50494,0.0440104,-0.411038,-0.83654],
[0.00991822,0.000701441,-0.0117267,-0.0185463,-0.00651771,0.000300882,-0.0105332,-0.0157532,-0.0290043,-0.0298136,-0.0270089,-0.0338438,-0.0422891,-0.0475227,-0.0688376,-0.0969972,-0.118332,-0.149331,-0.160628,-0.174346,-0.198937,-0.229174,-0.258218,-0.298552,-0.344138,-0.391761,-0.438599,-0.47699,-0.526289,-0.577219,-0.631397,-0.678339,-0.731756,-0.796905,-0.878636,-0.981398,-1.08421,-1.21372,-1.36431,-1.53396,-1.68751,-1.85892,-2.02515,-2.19873,-2.36956,-2.52549,-2.6568,-2.78976,-2.92077,-3.00809,-3.07034,-3.09815,-3.08707,-3.01232,-2.86905,-2.62389,-2.28489,-1.8922,-1.49731,-1.08357,-0.739083,-0.418,-0.149181,0.128288,0.387591,0.659201,0.913038,1.17187,1.38078,1.55154,1.60071,1.47895,1.23379,0.89366,0.484548,0.0333015,-0.447071,-0.969626,-1.43616,-1.82909,-2.18794],
[-0.00668519,0.0321222,0.0347521,0.0525987,0.0515605,0.0221773,0.0326826,0.0510695,0.0432022,0.0799874,0.0831576,0.0957905,0.0973887,0.0889968,0.0848086,0.0895618,0.0801105,0.076444,0.0711968,0.0564696,0.0575354,0.0670294,0.0728396,0.0623158,0.0607485,0.0918673,0.155695,0.252779,0.329331,0.394836,0.405476,0.425626,0.392454,0.328107,0.241545,0.137509,0.024971,-0.0770437,-0.213998,-0.367937,-0.538865,-0.781826,-1.01007,-1.24001,-1.4531,-1.62922,-1.78688,-1.9499,-2.08174,-2.22794,-2.34982,-2.4288,-2.43357,-2.35082,-2.21869,-1.99152,-1.66551,-1.22203,-0.667369,-0.0402341,0.596206,1.27702,1.92625,2.49921,2.9698,3.22732,3.35197,3.39419,3.26135,3.01667,2.74898,2.39811,2.04496,1.7007,1.20603,0.720204,0.277349,-0.137438,-0.520939,-0.865661,-1.18492],
[0.0234924,-0.0187281,-0.0650457,-0.0487051,-0.0282729,-0.0146493,-0.0269171,-0.0187421,-0.0323767,-0.0132931,-0.0269305,-0.0446624,-0.0828574,-0.115606,-0.127898,-0.157931,-0.209805,-0.276704,-0.33816,-0.418743,-0.519824,-0.678278,-0.851794,-1.04994,-1.21398,-1.4532,-1.74713,-2.04113,-2.36664,-2.69359,-3.0015,-3.32041,-3.65171,-3.92292,-4.17641,-4.4204,-4.63435,-4.77995,-4.90232,-4.99325,-5.01303,-5.06155,-5.05808,-5.06418,-5.01003,-5.00792,-4.95922,-4.90505,-4.83168,-4.79528,-4.70954,-4.58952,-4.38037,-3.96554,-3.40939,-2.69542,-1.84414,-0.937787,-0.0545251,0.827581,1.69207,2.60754,3.54109,4.43367,5.24962,5.97652,6.55395,7.04637,7.46612,7.73214,7.85124,7.80686,7.53027,7.1258,6.62776,5.99902,5.23815,4.48259,3.74196,3.02867,2.44036],
[0.0304898,0.0402553,0.0183019,0.0117972,-0.0158687,-0.0036628,-0.00692017,0.0150648,0.0272855,0.048473,0.0598918,0.0876158,0.0273095,0.00122308,-0.00693235,-0.0224331,-0.0101991,-0.0191785,-0.0910157,-0.135125,-0.204569,-0.270776,-0.319039,-0.413902,-0.504719,-0.61193,-0.703644,-0.801129,-0.928105,-1.06168,-1.18878,-1.32247,-1.36452,-1.42542,-1.4757,-1.49323,-1.49437,-1.47011,-1.39747,-1.36496,-1.32671,-1.29337,-1.29447,-1.3005,-1.30899,-1.38564,-1.50092,-1.58176,-1.7489,-1.89228,-2.10478,-2.33051,-2.47741,-2.51414,-2.50317,-2.33748,-2.12316,-1.80008,-1.3311,-0.772974,-0.223664,0.453605,1.18474,1.82386,2.43935,3.00646,3.4212,3.71649,3.91701,3.83201,3.65451,3.30435,2.86318,2.30864,1.67453,1.06659,0.551803,0.092194,-0.315513,-0.637368,-0.970114],
[-0.0130594,-0.0169499,-0.0695896,-0.055177,-0.0459009,-0.0231807,-0.0354618,-0.0373311,-0.0482555,-0.0446005,-0.0802121,-0.0858615,-0.18583,-0.260516,-0.348855,-0.458268,-0.517726,-0.554809,-0.672021,-0.737074,-0.852059,-1.08132,-1.35027,-1.66218,-1.98882,-2.2761,-2.62872,-3.01891,-3.38973,-3.65926,-3.94684,-4.2091,-4.54889,-4.94049,-5.21333,-5.39209,-5.54129,-5.64473,-5.75085,-5.887,-5.86322,-5.83131,-5.77936,-5.64864,-5.45777,-5.31788,-5.20388,-5.25412,-5.35871,-5.54346,-5.67966,-5.69555,-5.51213,-5.04821,-4.20626,-3.29727,-2.16033,-0.866243,0.507329,2.15801,3.95643,5.82721,7.80452,9.59422,11.0179,12.1938,12.8901,13.2734,13.3539,13.1311,12.7486,12.1387,11.1678,10.1393,8.96668,7.86492,7.04728,6.27582,5.64742,4.75309,3.84225],
[0.0214452,-0.00762488,-0.0266428,-0.0338268,-0.0305014,-0.00992306,-0.00349063,0.00740323,-0.0165533,-0.0214698,-0.0231119,-0.00882743,-0.0366783,-0.0423622,-0.103689,-0.151222,-0.209422,-0.275014,-0.373837,-0.440132,-0.557547,-0.691768,-0.841325,-1.02473,-1.23335,-1.44943,-1.70944,-2.01941,-2.31947,-2.63936,-2.96627,-3.29301,-3.6261,-3.98774,-4.28639,-4.54847,-4.73678,-4.85567,-4.93217,-5.00137,-5.03122,-5.04735,-5.01463,-4.973,-4.92282,-4.8817,-4.87247,-4.87859,-4.90639,-5.00444,-5.10934,-5.16611,-5.12203,-4.848,-4.33452,-3.57591,-2.55581,-1.34135,-0.0146699,1.46484,3.09896,4.85769,6.7284,8.47811,9.90299,10.8161,11.2682,11.3217,11.1268,10.6665,10.0288,9.15077,8.2064,7.22176,6.2102,5.2421,4.35001,3.51314,2.89584,2.34722,1.82543],
[-0.0163512,0.0264012,0.0195467,-0.00272336,-0.00919175,-0.00230755,0.0301616,0.00789479,-0.0315146,0.00463946,0.0608332,0.066026,-0.0362365,-0.212976,-0.341059,-0.382353,-0.397757,-0.448157,-0.561336,-0.761453,-0.986988,-1.22422,-1.5101,-1.84989,-2.25337,-2.68101,-3.1286,-3.60036,-4.04781,-4.4995,-4.98535,-5.45664,-5.86453,-6.17284,-6.39048,-6.57068,-6.6954,-6.68839,-6.58064,-6.41697,-6.2497,-6.07773,-5.86556,-5.63905,-5.42349,-5.27939,-5.20226,-5.11111,-4.98906,-4.88244,-4.76716,-4.54467,-3.98679,-2.96628,-1.61532,-0.0870679,1.51445,3.20434,5.1277,7.31925,9.61018,11.8018,13.677,14.9857,15.6206,15.6747,15.3533,14.8492,14.1853,13.2218,11.9573,10.5432,9.12874,7.79794,6.57294,5.42692,4.41265,3.5813,2.92349,2.46684,2.10365],
[-0.04219,0.015641,0.0281915,0.0778767,0.0648455,0.0526988,-0.0357253,0.0130954,0.0886264,0.089588,0.0377819,0.0753163,-0.00234925,-0.0343397,0.0396627,0.00817105,-0.0210473,-0.0683863,-0.162833,-0.346408,-0.380721,-0.430336,-0.477451,-0.558962,-0.672423,-0.908003,-1.10993,-1.35858,-1.60399,-1.78361,-2.00609,-2.17697,-2.3646,-2.61604,-2.82616,-3.00682,-3.22564,-3.4487,-3.69868,-3.95699,-4.10069,-4.24489,-4.29071,-4.34665,-4.29995,-4.11402,-3.88986,-3.67099,-3.38874,-3.03193,-2.59464,-2.03901,-1.46447,-0.841282,-0.0897227,0.771901,1.65757,2.63105,3.65223,4.55361,5.44608,6.16089,6.72934,7.18653,7.51973,7.54372,7.25701,6.74324,6.01873,5.23159,4.4411,3.48002,2.39707,1.48525,0.380248,-0.508884,-1.35651,-2.19415,-3.05739,-3.80249,-4.38341],
[-0.0243059,0.0216324,0.0929879,0.105976,0.0669451,0.0175656,0.0332745,0.0582116,0.0634792,0.0699605,0.0422232,-0.0221469,-0.0369495,-0.11843,-0.216955,-0.244174,-0.253817,-0.359119,-0.475223,-0.652114,-0.869032,-1.03363,-1.20546,-1.4568,-1.68047,-1.93492,-2.21337,-2.49086,-2.7591,-3.08421,-3.35109,-3.67977,-3.98547,-4.31708,-4.58037,-4.82886,-5.02109,-5.10861,-5.23301,-5.3696,-5.37553,-5.28178,-5.13376,-4.93539,-4.89281,-4.83977,-4.81801,-4.82304,-4.90016,-4.98353,-5.08777,-5.03608,-4.82932,-4.39663,-3.77132,-2.84414,-1.83326,-0.731452,0.567199,2.05956,3.71962,5.54451,7.30533,8.86418,10.1499,11.0108,11.5685,11.88,11.7936,11.4888,10.8411,9.87694,8.8934,7.96591,7.0496,6.27877,5.49144,4.75393,3.91751,3.14674,2.61781],
[-0.00234,-0.00142,-0.0323,-0.0317,0.00186,0.018,0.0277,0.032,0.0231,0.00452,-0.0157,-0.0776,-0.137,-0.205,-0.244,-0.298,-0.387,-0.488,-0.612,-0.77,-0.948,-1.19,-1.49,-1.82,-2.19,-2.61,-3.07,-3.6,-4.16,-4.77,-5.39,-5.95,-6.38,-6.73,-6.98,-7.19,-7.34,-7.36,-7.3,-7.19,-7.02,-6.83,-6.63,-6.41,-6.29,-6.2,-6.15,-6.13,-6.17,-6.27,-6.39,-6.4,-6.15,-5.6,-4.7,-3.52,-2.09,-0.465,1.35,3.44,5.87,8.51,11.1,13.4,15.1,16.2,16.8,17.1,17.1,16.8,16.4,15.7,14.9,13.9,12.7,11.5,10.3,9.1,7.98,6.95,5.99],
[0.0308173,0.0230442,0.0198572,0.0182808,0.0372441,0.0759209,0.0871364,0.0555436,0.0308752,0.00767062,-0.00453208,0.00196338,0.0114385,0.021225,0.0233501,-0.0406602,-0.154933,-0.25599,-0.376479,-0.534185,-0.723549,-0.9778,-1.27587,-1.56513,-1.8364,-2.14081,-2.5194,-2.99534,-3.50772,-3.97509,-4.37594,-4.75008,-5.14586,-5.50881,-5.82306,-6.08133,-6.24404,-6.34104,-6.39734,-6.43141,-6.45765,-6.45737,-6.39942,-6.30682,-6.2397,-6.19403,-6.1624,-6.21188,-6.34142,-6.48236,-6.55946,-6.49482,-6.25768,-5.83933,-5.12794,-4.11972,-2.94643,-1.62569,-0.122277,1.57874,3.46262,5.38973,7.21821,8.83834,10.0971,10.896,11.2629,11.3154,11.0869,10.6002,9.84682,8.87109,7.79169,6.65584,5.5495,4.52283,3.579,2.71883,1.94208,1.27283,0.683351],
[0.0757209,-0.00981236,-0.0796268,-0.0873545,-0.0424019,0.0200409,0.0489052,0.0407014,0.0176942,-0.00475512,-0.0507732,-0.127943,-0.1799,-0.166287,-0.122175,-0.111465,-0.185221,-0.318065,-0.418219,-0.466627,-0.548854,-0.685956,-0.859029,-1.06083,-1.26512,-1.48,-1.7227,-1.97298,-2.23818,-2.58739,-2.97911,-3.30967,-3.61365,-3.88854,-4.12429,-4.36329,-4.5167,-4.57676,-4.63088,-4.66694,-4.7025,-4.7,-4.64116,-4.57525,-4.51641,-4.4654,-4.41347,-4.34874,-4.23305,-4.00974,-3.66947,-3.19061,-2.58704,-1.90493,-1.06219,-0.0522734,1.08875,2.31797,3.57172,4.80549,6.06753,7.33659,8.4724,9.41181,10.0759,10.427,10.4633,10.1642,9.58271,8.84998,8.04074,7.15717,6.15815,5.03567,3.90059,2.97355,2.20598,1.49333,0.832017,0.221999,-0.210266],
[-0.0296735,-0.0240353,-0.0178808,-0.00500642,0.00743114,-0.012677,-0.0200865,-0.0203592,-0.0147318,0.00300319,0.0253717,0.00859656,-0.00141316,-0.0144119,-0.0257329,-0.0223195,-0.0198229,-0.0295583,-0.034748,-0.0630613,-0.0902449,-0.115628,-0.147102,-0.175251,-0.205082,-0.242756,-0.300759,-0.356496,-0.407757,-0.471798,-0.530835,-0.607834,-0.704807,-0.804634,-0.901689,-0.992937,-1.07627,-1.16252,-1.2419,-1.29981,-1.34242,-1.34853,-1.34024,-1.31971,-1.31616,-1.26253,-1.1775,-1.05994,-0.910585,-0.742689,-0.571889,-0.342056,-0.0741652,0.253778,0.62747,1.02505,1.42068,1.81226,2.15224,2.42682,2.65418,2.81266,2.87522,2.85664,2.71672,2.50322,2.23295,1.92507,1.6046,1.26521,0.952414,0.656572,0.371238,0.121865,-0.0826638,-0.30853,-0.480974,-0.721685,-0.941395,-1.18252,-1.37137],
[-0.0577359,0.00353969,-0.00495019,0.00956822,-0.0134619,-0.0067086,-0.00228141,0.0287103,-0.00623769,-0.0307777,-0.0467129,-0.0609084,-0.0276025,-0.0202901,-0.0283363,-0.055981,-0.0794854,-0.0993145,-0.139029,-0.206867,-0.246527,-0.2886,-0.345194,-0.42117,-0.504799,-0.601385,-0.694059,-0.839343,-1.0165,-1.16473,-1.30924,-1.40437,-1.49169,-1.59968,-1.73817,-1.88765,-2.03236,-2.16271,-2.27781,-2.40394,-2.51956,-2.6055,-2.66875,-2.66527,-2.61994,-2.52495,-2.36882,-2.14244,-1.8664,-1.51184,-1.10648,-0.596027,0.00248504,0.674,1.41058,2.19508,2.98686,3.70681,4.37973,4.93069,5.35346,5.68321,5.93314,6.07979,6.10802,5.94015,5.57261,4.99857,4.35038,3.78961,3.16804,2.54577,1.78562,1.01702,0.289148,-0.426744,-1.04171,-1.71742,-2.39142,-2.97554,-3.49794],
[0.045902,0.0374928,-0.00276251,-0.049222,-0.0798912,-0.0816122,-0.015534,0.0239411,0.0820398,0.0989642,0.132759,0.112865,0.0221676,-0.050564,-0.158656,-0.206638,-0.220963,-0.249248,-0.330499,-0.391883,-0.57723,-0.685042,-0.832681,-1.12004,-1.33907,-1.59574,-1.86437,-2.15455,-2.45992,-2.88603,-3.20725,-3.63604,-3.98375,-4.40186,-4.72508,-4.97409,-5.18188,-5.38187,-5.49863,-5.57531,-5.5899,-5.56269,-5.58678,-5.57065,-5.54491,-5.4765,-5.46812,-5.50063,-5.61447,-5.68612,-5.76675,-5.7618,-5.68893,-5.42192,-4.92217,-4.06542,-2.99846,-1.6707,-0.220184,1.54793,3.58492,5.88301,8.18199,10.2292,11.7166,12.7154,13.2131,13.2396,12.9007,12.238,11.2871,10.0755,8.87723,7.71593,6.61656,5.73451,5.02883,4.28738,3.27973,2.78732,2.40037],
[0.0435697,0.00144791,-0.0200182,-0.0532137,-0.0557667,-0.0759999,-0.0670536,-0.0643148,-0.0387859,-0.0255153,-0.0119596,-0.0306222,-0.0779126,-0.15402,-0.195404,-0.24517,-0.297472,-0.350021,-0.434272,-0.554027,-0.687845,-0.862512,-1.054,-1.27076,-1.52979,-1.80695,-2.13225,-2.48127,-2.84922,-3.24563,-3.67315,-4.0832,-4.48812,-4.85342,-5.15078,-5.41085,-5.62256,-5.75495,-5.84025,-5.89637,-5.93682,-5.97347,-5.94689,-5.85523,-5.76532,-5.68436,-5.62853,-5.62915,-5.67874,-5.77441,-5.9504,-6.09409,-6.09064,-5.88228,-5.41686,-4.6412,-3.65576,-2.52694,-1.18938,0.354557,2.11119,4.11084,6.31789,8.52218,10.5992,12.3155,13.5761,14.4507,14.9564,15.0081,14.7084,14.0856,13.1942,12.1392,11.0604,9.90081,8.94632,8.02546,7.14853,5.89262,4.59285],
[-0.079809,-0.0604967,-0.058761,-0.0426616,-0.0272771,-0.0772329,-0.131238,-0.164793,-0.189915,-0.184187,-0.154719,-0.114538,-0.114428,-0.161518,-0.224358,-0.281637,-0.315623,-0.323607,-0.35742,-0.443394,-0.517858,-0.597905,-0.698731,-0.796898,-0.866863,-0.968061,-1.11143,-1.31614,-1.54497,-1.73916,-1.87816,-1.98814,-2.10217,-2.24368,-2.40906,-2.54076,-2.60612,-2.68328,-2.75287,-2.7942,-2.84235,-2.88145,-2.8757,-2.92528,-2.9735,-2.95066,-2.97606,-3.007,-3.05775,-3.14044,-3.16976,-3.06102,-2.91621,-2.66996,-2.3208,-1.89723,-1.35335,-0.674622,0.0917738,0.951042,1.85731,2.71999,3.50902,4.18977,4.68458,5.03256,5.24677,5.30496,5.20164,4.95103,4.57868,4.14364,3.64193,3.07051,2.51535,2.03185,1.66243,1.3799,1.04834,0.711966,0.462741],
[-0.0467027,-0.0210813,-0.025955,0.00470639,0.0190464,0.0620042,0.0295831,0.0366144,0.0564643,0.0859808,0.0717734,0.000627293,-0.096685,-0.182622,-0.257474,-0.328981,-0.427539,-0.605279,-0.796958,-0.985806,-1.21993,-1.50536,-1.78783,-2.1926,-2.66204,-3.15132,-3.70942,-4.28833,-4.80769,-5.3997,-5.99502,-6.5978,-7.09625,-7.50617,-7.76755,-7.97115,-8.13457,-8.24398,-8.2006,-8.15078,-8.03742,-7.93499,-7.85465,-7.71852,-7.52598,-7.39676,-7.33752,-7.35293,-7.4265,-7.52352,-7.56029,-7.37264,-6.89115,-6.00052,-4.67797,-3.06543,-1.14775,1.07205,3.53009,6.26814,9.09723,11.9824,14.4945,16.374,17.388,17.6994,17.355,16.7277,15.6817,14.4517,13.0468,11.5317,10.0705,8.74962,7.53773,6.45676,5.58175,4.74205,4.07943,3.58858,3.09104],
[0.0506642,0.0423635,0.0606642,0.0290418,-0.0375204,-0.0626404,-0.0213632,-0.0630112,-0.0321333,-0.0294555,-0.0697113,-0.0982951,-0.0923037,-0.16368,-0.218831,-0.278002,-0.364589,-0.485737,-0.643646,-0.848277,-1.07577,-1.32925,-1.63588,-1.95877,-2.34879,-2.76357,-3.22906,-3.69788,-4.24351,-4.72256,-5.20383,-5.61272,-5.91808,-6.15309,-6.41742,-6.58043,-6.66877,-6.70217,-6.62234,-6.56769,-6.49389,-6.36811,-6.23847,-6.10442,-6.00917,-5.93536,-5.89023,-5.89985,-6.03276,-6.17017,-6.26122,-6.11166,-5.74109,-5.00541,-3.90248,-2.49262,-0.72265,1.28388,3.43613,5.87925,8.57971,11.2182,13.7604,15.7225,16.881,17.2017,16.93,16.0567,15.0651,13.9889,12.8791,11.602,10.5851,9.40741,8.47779,7.65923,6.83046,6.01342,5.23925,4.23338,3.46441],
[0.682943,0.720151,0.736581,0.751917,0.788036,0.780408,0.716997,0.70827,0.656881,0.604395,0.593477,0.533328,0.484114,0.397699,0.324407,0.260956,0.196409,0.0968397,-0.1242,-0.416385,-0.694363,-0.919835,-1.14641,-1.32813,-1.71126,-2.13381,-2.49291,-2.80386,-3.11703,-3.41381,-3.85292,-4.2603,-4.6513,-5.0259,-5.40053,-5.78942,-6.15315,-6.39426,-6.49304,-6.52612,-6.52854,-6.54191,-6.54981,-6.60151,-6.56888,-6.53407,-6.49486,-6.56629,-6.71659,-6.88772,-6.96901,-6.91887,-6.60249,-6.0604,-5.24763,-4.10172,-2.79247,-1.28698,0.364356,2.24813,4.40381,6.77225,9.25155,11.7212,13.8084,15.4464,16.7207,17.5083,18.0044,18.1158,17.6515,16.7598,15.573,14.2141,12.843,11.6592,10.4315,9.38358,8.47594,7.65928,6.91602],
[0.00754245,-0.00027169,-0.0271516,-0.0174814,0.0191852,0.0522355,0.0430816,0.0057951,-0.0071547,0.0172475,0.0424265,0.0258251,-0.029213,-0.0761221,-0.103876,-0.139223,-0.194869,-0.29925,-0.428477,-0.552741,-0.696038,-0.891981,-1.14842,-1.45828,-1.80411,-2.1699,-2.56196,-2.99519,-3.45308,-3.91936,-4.37267,-4.76933,-5.10816,-5.39579,-5.63346,-5.84969,-6.02226,-6.11864,-6.15374,-6.114,-6.02952,-5.91634,-5.77109,-5.64581,-5.59987,-5.62967,-5.67238,-5.71013,-5.77217,-5.88619,-6.00882,-5.99896,-5.7234,-5.13074,-4.25258,-3.14593,-1.85046,-0.384774,1.2458,3.06128,5.04935,7.09182,8.96533,10.4748,11.5172,12.1635,12.5405,12.6649,12.5289,12.2156,11.7859,11.2585,10.6498,9.95369,9.227,8.53714,7.93375,7.36497,6.78479,6.18819,5.64831],
[-0.000437059,0.0126741,0.0231619,0.0353968,0.00917653,-0.0589892,-0.0685989,-0.0694693,-0.0790781,-0.0703369,-0.0773231,-0.079067,-0.03975,-0.0441161,-0.0607111,-0.08604,-0.127963,-0.180362,-0.245857,-0.336673,-0.347136,-0.34712,-0.342738,-0.365425,-0.433512,-0.55747,-0.704119,-0.842895,-0.897853,-0.972009,-1.01037,-1.07142,-1.14818,-1.22667,-1.26066,-1.31558,-1.37311,-1.39486,-1.40963,-1.47762,-1.50634,-1.5307,-1.56465,-1.5323,-1.45546,-1.34462,-1.17185,-0.961596,-0.786247,-0.566437,-0.214966,0.153041,0.526247,0.981382,1.40944,1.93163,2.59936,3.25308,3.82653,4.30056,4.57406,4.72289,4.6451,4.4514,4.12525,3.73464,3.33187,2.99275,2.63884,2.37123,2.14024,1.89359,1.64521,1.35853,0.956895,0.522195,0.0753426,-0.482953,-1.04381,-1.59677,-2.11311],
[-0.0899312,-0.0990272,-0.0594102,-0.0683548,-0.0785901,-0.0797002,-0.0666776,-0.0643341,-0.0658429,-0.0585542,-0.0638703,-0.0735649,-0.0979866,-0.157291,-0.168855,-0.125016,-0.118467,-0.100745,-0.109104,-0.166092,-0.184232,-0.154636,-0.163635,-0.183865,-0.212152,-0.276747,-0.345923,-0.379294,-0.436859,-0.498111,-0.600877,-0.728171,-0.878218,-1.04379,-1.205,-1.31171,-1.4736,-1.59165,-1.70757,-1.86129,-2.03228,-2.14958,-2.27457,-2.37073,-2.41584,-2.49302,-2.5849,-2.67448,-2.74681,-2.89575,-3.02729,-3.19618,-3.36007,-3.53853,-3.63717,-3.70103,-3.64224,-3.60084,-3.55122,-3.4684,-3.28687,-2.97073,-2.4593,-1.83352,-1.08807,-0.331417,0.371327,1.06738,1.6743,2.29495,2.79263,3.14106,3.28707,3.35718,3.22993,3.09601,2.76319,2.52521,2.1194,1.85253,1.69522],
[-0.00120238,-0.0188201,-0.0522586,-0.0753936,-0.0635176,-0.0544793,-0.0739225,-0.086415,-0.0998439,-0.118684,-0.114687,-0.122996,-0.152,-0.171673,-0.204438,-0.233913,-0.287973,-0.369458,-0.493003,-0.604954,-0.726283,-0.834952,-1.00157,-1.16868,-1.4088,-1.64812,-1.91466,-2.22434,-2.57591,-2.92241,-3.32359,-3.77421,-4.21048,-4.64804,-4.9866,-5.26557,-5.43257,-5.55209,-5.59765,-5.60855,-5.55539,-5.56572,-5.49278,-5.42395,-5.37215,-5.32147,-5.31782,-5.3581,-5.38572,-5.45651,-5.54448,-5.57414,-5.42176,-5.02758,-4.3571,-3.45233,-2.336,-1.06905,0.396507,2.01557,3.71549,5.47078,7.12816,8.46491,9.40825,9.8883,9.97904,9.76012,9.305,8.68668,7.84431,6.90438,5.94096,4.95557,4.09156,3.28937,2.51399,1.83698,1.37885,1.0279,0.695599],
[0.0233481,0.0166243,-0.00456315,-0.0155276,0.013834,0.0510844,0.0262857,0.0124469,-0.0367916,-0.00961585,0.0408226,0.0941476,0.0600032,0.019599,-0.0761923,-0.177768,-0.201717,-0.217962,-0.314391,-0.400953,-0.479797,-0.674637,-0.819829,-0.997687,-1.25512,-1.51399,-1.77966,-2.10477,-2.42159,-2.8361,-3.36157,-3.88238,-4.39914,-4.8341,-5.2227,-5.56118,-5.77539,-5.86481,-5.91407,-5.87503,-5.86233,-5.80672,-5.62898,-5.45347,-5.38288,-5.29365,-5.28648,-5.32259,-5.31926,-5.3886,-5.49359,-5.56445,-5.52898,-5.30908,-4.80611,-4.02626,-3.02747,-1.9198,-0.558115,0.898763,2.54619,4.33851,6.13373,7.69541,9.0113,9.79882,10.225,10.3147,10.2516,9.98098,9.53121,8.9037,8.26326,7.42989,6.65779,5.87791,5.0627,4.33259,3.76638,3.17696,2.72648],
[-0.0124574,-0.00426044,0.0120137,0.0254749,0.0288966,0.00929016,-0.00487496,-0.0193157,-0.0276337,-0.0412898,-0.053497,-0.0680504,-0.0868632,-0.115237,-0.156149,-0.195904,-0.245359,-0.316346,-0.399656,-0.518329,-0.656734,-0.800712,-0.964591,-1.15305,-1.38442,-1.64408,-1.92391,-2.21185,-2.52897,-2.87271,-3.27269,-3.67867,-4.06357,-4.40082,-4.69506,-4.93073,-5.10789,-5.24703,-5.35688,-5.42454,-5.4266,-5.37999,-5.28005,-5.17076,-5.08474,-5.01956,-4.97535,-4.97041,-4.972,-5.01008,-5.05672,-5.04932,-4.89887,-4.54527,-3.91931,-3.09888,-2.12781,-1.04227,0.175151,1.54227,3.08052,4.78034,6.57894,8.3005,9.83909,11.0453,11.8748,12.3483,12.4933,12.3157,11.9228,11.3217,10.6401,9.94429,9.27379,8.6578,8.1458,7.6934,7.31617,7.0249,6.77405],
[0.00340146,-0.0341361,-0.0536474,-0.071165,-0.0699,-0.0572119,-0.0521147,-0.0316963,0.0218183,0.0252585,-0.00383215,-0.0822267,-0.146569,-0.206523,-0.237453,-0.286833,-0.375315,-0.490966,-0.589846,-0.732448,-0.883997,-1.06237,-1.2778,-1.50245,-1.77644,-2.11334,-2.47722,-2.87649,-3.29402,-3.69045,-4.07368,-4.46639,-4.84194,-5.21994,-5.59727,-5.92897,-6.16074,-6.36658,-6.51439,-6.56977,-6.56939,-6.50539,-6.42015,-6.34644,-6.30213,-6.203,-6.13812,-6.13102,-6.20848,-6.34197,-6.54498,-6.64941,-6.61592,-6.37916,-5.81522,-4.95217,-3.9122,-2.65128,-1.20648,0.445234,2.33421,4.47496,6.77912,9.10775,11.2249,12.9117,14.0748,14.8072,15.1575,15.1497,14.846,14.233,13.4085,12.405,11.2529,10.073,8.91916,7.84754,7.1015,6.45185,5.80798],
[0.00888393,-0.00230244,0.00144985,0.0335459,0.0420456,0.0377629,0.00169354,0.00744145,-0.0165619,-0.0133432,0.00275776,-0.022601,-0.0523588,-0.0577089,-0.0648725,-0.0638532,-0.150508,-0.263246,-0.368424,-0.492229,-0.611219,-0.761378,-0.992596,-1.25957,-1.51833,-1.76818,-2.04655,-2.32507,-2.59947,-2.97854,-3.38193,-3.70192,-4.06576,-4.34735,-4.62073,-4.84363,-5.01355,-5.10059,-5.20718,-5.20939,-5.18783,-5.08203,-4.99344,-4.87694,-4.82064,-4.73328,-4.66129,-4.65119,-4.66407,-4.70641,-4.84911,-4.96565,-4.99563,-4.78492,-4.33406,-3.63728,-2.85785,-1.91174,-0.948063,0.212301,1.54879,3.13349,4.83138,6.6007,8.14272,9.438,10.341,10.9489,11.2371,11.2058,10.8454,10.1645,9.13461,8.09888,7.15604,6.09291,5.10297,4.25523,3.428,2.65688,1.94178],
[0.000624339,0.0109413,-0.00821735,0.0106303,0.0243897,-0.00102381,-0.0332325,-0.0410895,-0.0458306,-0.0428202,-0.0457049,-0.0985086,-0.131892,-0.143022,-0.144048,-0.1909,-0.25006,-0.340147,-0.425723,-0.508371,-0.641286,-0.803761,-1.01323,-1.27439,-1.55493,-1.85746,-2.20417,-2.55133,-2.96194,-3.36684,-3.74152,-4.11297,-4.4388,-4.76328,-5.04659,-5.26519,-5.40928,-5.48559,-5.48735,-5.5162,-5.47326,-5.4535,-5.40375,-5.33786,-5.26945,-5.27392,-5.28626,-5.33781,-5.41763,-5.56288,-5.72302,-5.84798,-5.82531,-5.5738,-5.07459,-4.374,-3.4905,-2.41803,-1.18624,0.257288,1.93984,3.81014,5.72842,7.50375,8.89613,9.89794,10.4642,10.6649,10.5625,10.151,9.43634,8.57703,7.63149,6.70985,5.84127,5.093,4.31689,3.63068,3.04863,2.52299,2.07782],
[-0.0129124,-0.036952,-0.0186863,-0.0140419,-0.00702641,-1.28902e-05,0.00926722,0.0131837,0.0516324,0.046678,0.0116325,-0.00475285,0.00462341,-0.00475157,-0.0186561,-0.0600544,-0.101029,-0.0820713,-0.0512797,-0.0573456,-0.0768922,-0.1413,-0.196635,-0.268209,-0.350154,-0.420146,-0.525084,-0.669167,-0.784321,-0.957832,-1.1204,-1.29937,-1.53942,-1.75506,-1.94671,-2.11181,-2.23856,-2.36928,-2.46413,-2.52302,-2.54116,-2.49648,-2.46587,-2.35268,-2.26291,-2.21377,-2.10046,-1.97579,-1.8693,-1.67294,-1.49959,-1.24901,-0.945627,-0.611383,-0.277125,0.0467981,0.401769,0.868159,1.28835,1.74425,2.24425,2.76314,3.41281,4.10473,4.69028,5.14034,5.45333,5.55443,5.55123,5.36379,5.02247,4.59219,4.06394,3.45032,2.93686,2.35235,1.68527,1.03369,0.545362,0.0673711,-0.213468],
[0.0593053,0.0425652,0.0407661,0.0494946,0.0348879,0.0323182,0.0537049,0.0808557,0.0643421,0.0539806,0.00587028,-0.0324623,-0.0660906,-0.030269,-0.00905128,0.0236833,0.0244154,-0.028707,-0.0857986,-0.143375,-0.216926,-0.259646,-0.344466,-0.496164,-0.61633,-0.785599,-0.930947,-1.02101,-1.1356,-1.2312,-1.33299,-1.50148,-1.63762,-1.82491,-2.01689,-2.18572,-2.36793,-2.55034,-2.68007,-2.81143,-2.93967,-3.04571,-3.14469,-3.23777,-3.31814,-3.37399,-3.40516,-3.46399,-3.51339,-3.59612,-3.61802,-3.67486,-3.64921,-3.6466,-3.51185,-3.31853,-2.98055,-2.62866,-2.19388,-1.61714,-0.918352,-0.176882,0.608259,1.37807,2.12204,2.91417,3.60937,4.28692,4.75159,5.02664,5.21508,5.29127,5.16252,4.94487,4.23914,3.89945,3.05926,2.26252,1.90862,1.32979,0.563255]]

ref_lincomb_ss = [[0.4240549828178694,0.032989690721649485,0.010996563573883162,0.005498281786941581,0.12096219931271478,0.10309278350515463,0.0,0.005498281786941581,0.29690721649484536],
[0.27808676307007785,0.03225806451612903,0.02314884406096865,0.06250187896461533,0.0967741935483871,0.13236929922135707,0.0,0.025583982202447165,0.3492769744160178],
[0.045454545454545456,0.0,0.11636363636363635,0.24727272727272728,0.03409090909090909,0.1590909090909091,0.0,0.0,0.3977272727272727],
[0.09224318658280922,0.025157232704402517,0.14538784067085953,0.1606918238993711,0.016771488469601678,0.12368972746331237,0.0,0.027253668763102725,0.4088050314465409],
[0.07331975560081466,0.06109979633401222,0.1395924181275224,0.15979658390913748,0.020366598778004074,0.14867617107942974,0.0,0.03054989816700611,0.3665987780040733],
[0.3112449799196787,0.07228915662650602,0.012048192771084338,0.008032128514056224,0.09437751004016064,0.12650602409638553,0.0,0.018072289156626505,0.357429718875502],
[0.10201834862385321,0.03547400611620795,0.07309909211446879,0.24274188647880338,0.047828746177370034,0.11314984709480122,0.0012232415902140672,0.025198776758409785,0.35926605504587156],
[0.08552631578947369,0.06578947368421052,0.11816135734072021,0.3094702216066482,0.019736842105263157,0.07894736842105263,0.0,0.0,0.3223684210526316],
[0.7200992555831266,0.020843672456575684,0.0,0.0,0.0,0.07791563275434243,0.030272952853598014,0.0,0.15086848635235733],
[0.5753424657534246,0.0,0.0273972602739726,0.0273972602739726,0.0,0.0821917808219178,0.0,0.0,0.2876712328767123],
[0.2742340532395781,0.04369663485685585,0.051839927874882945,0.11089236745409747,0.004018081366147665,0.12656956303365144,0.020090406830738324,0.020090406830738324,0.34856855851330987],
[0.08627450980392157,0.03431372549019608,0.1120471367354889,0.1771685495390209,0.052941176470588235,0.14215686274509803,0.0,0.022549019607843137,0.37254901960784315],
[0.5720823798627003,0.02288329519450801,0.004576659038901602,0.009153318077803204,0.0,0.09839816933638444,0.02288329519450801,0.011441647597254004,0.2585812356979405],
[0.0,0.0379746835443038,0.07081575246132207,0.37644163150492266,0.012658227848101266,0.11814345991561181,0.0,0.008438818565400843,0.3755274261603376],
[0.41346153846153844,0.0,0.0,0.0,0.0,0.17307692307692307,0.0,0.019230769230769232,0.3942307692307692],
[0.0,0.07734806629834254,0.10162122996105423,0.26301965401684635,0.0718232044198895,0.08287292817679558,0.0,0.011049723756906077,0.39226519337016574],
[0.028735632183908046,0.06321839080459771,0.12454966546577455,0.2547606793618117,0.08045977011494253,0.08045977011494253,0.0,0.022988505747126436,0.3448275862068966],
[0.025936599423631124,0.06340057636887608,0.11384337070615705,0.2665600875070994,0.07204610951008646,0.08069164265129683,0.0,0.023054755043227664,0.35446685878962536],
[0.0,0.06358381502890173,0.10244553134726546,0.26749666518452647,0.06358381502890173,0.09826589595375723,0.0,0.011560693641618497,0.3930635838150289],
[0.05714285714285714,0.06285714285714286,0.12190476190476192,0.2438095238095238,0.08,0.06285714285714286,0.0,0.022857142857142857,0.3485714285714286],
[0.0,0.08092485549132948,0.09420403062021555,0.2930792063740041,0.09826589595375723,0.08670520231213873,0.0,0.023121387283236993,0.3236994219653179],
[0.3920792079207921,0.047524752475247525,0.007920792079207921,0.015841584158415842,0.16237623762376238,0.08712871287128712,0.019801980198019802,0.015841584158415842,0.2514851485148515],
[0.3357664233576642,0.13138686131386862,0.0,0.0,0.18248175182481752,0.0948905109489051,0.0,0.0,0.25547445255474455],
[0.2548262548262548,0.03474903474903475,0.06446706446706446,0.15947115947115947,0.03861003861003861,0.08108108108108109,0.0,0.023166023166023165,0.3436293436293436],
[0.05416666666666667,0.05416666666666667,0.13234374999999998,0.18848958333333335,0.020833333333333332,0.17083333333333334,0.0,0.03333333333333333,0.3458333333333333],
[0.2633390705679862,0.07228915662650602,0.0402065404475043,0.08543889845094664,0.06712564543889846,0.12220309810671257,0.008605851979345954,0.04130808950086059,0.29948364888123924],
[0.4374240583232078,0.046172539489671933,0.04284970367247749,0.034914573362759445,0.07533414337788578,0.0959902794653706,0.02551640340218712,0.004860267314702308,0.23693803159173754],
[0.3470790378006873,0.08247422680412371,0.01348136399682791,0.04493787998942639,0.12714776632302405,0.0859106529209622,0.03436426116838488,0.010309278350515464,0.2542955326460481],
[0.6695652173913044,0.0782608695652174,0.0,0.0,0.0,0.0817391304347826,0.02608695652173913,0.0,0.1443478260869565],
[0.6861063464837049,0.030874785591766724,0.0,0.0,0.0,0.09433962264150944,0.008576329331046312,0.0,0.18010291595197256],
[0.039423805913570885,0.03184230477634572,0.10851288219872005,0.3077115302349418,0.03411675511751327,0.10386656557998483,0.0,0.018953752843062926,0.3555724033358605],
[0.2948905109489051,0.021897810218978103,0.0558809941790631,0.06674674304721427,0.05401459854014599,0.17226277372262774,0.014598540145985401,0.00291970802919708,0.3167883211678832],
[0.017429193899782137,0.026143790849673203,0.07722882064016368,0.3911807654164812,0.008714596949891068,0.10893246187363835,0.0,0.013071895424836602,0.3572984749455338],
[0.6259541984732825,0.05343511450381679,0.0,0.0,0.0,0.03816793893129771,0.0,0.015267175572519083,0.26717557251908397],
[0.3023255813953488,0.10077519379844961,0.024806201550387597,0.037209302325581395,0.0,0.24031007751937986,0.0,0.046511627906976744,0.24806201550387597],
[0.7124183006535948,0.0,0.0,0.0,0.0,0.11764705882352941,0.0392156862745098,0.0,0.13071895424836602],
[0.7417218543046358,0.039735099337748346,0.0,0.0,0.0,0.09271523178807947,0.0,0.0,0.12582781456953643],
[0.33126934984520123,0.018575851393188854,0.018575851393188854,0.018575851393188854,0.1393188854489164,0.10526315789473684,0.04953560371517028,0.006191950464396285,0.3126934984520124],
[0.2951251646903821,0.023715415019762844,0.06736552672280463,0.20931431517442856,0.04743083003952569,0.12384716732542819,0.0,0.005270092226613966,0.22793148880105402],
[0.25801749271137026,0.030612244897959183,0.052478134110787174,0.06559766763848396,0.048104956268221574,0.1457725947521866,0.014577259475218658,0.007288629737609329,0.37755102040816324],
[0.23113207547169812,0.02830188679245283,0.07004002287021155,0.09505431675242997,0.014150943396226415,0.1320754716981132,0.0,0.04245283018867924,0.3867924528301887],
[0.008676789587852495,0.03253796095444685,0.07373704942167787,0.39047119352843057,0.008676789587852495,0.1193058568329718,0.0,0.008676789587852495,0.3579175704989154],
[0.08806818181818182,0.04261363636363636,0.007575757575757577,0.0037878787878787876,0.3096590909090909,0.07954545454545454,0.0,0.019886363636363636,0.44886363636363635],
[0.10810810810810811,0.0972972972972973,0.11183597390493942,0.2124883504193849,0.062162162162162166,0.11351351351351352,0.0,0.010810810810810811,0.28378378378378377],
[0.3226381461675579,0.026737967914438502,0.04401480872069108,0.14671602906897024,0.06060606060606061,0.11764705882352941,0.0142602495543672,0.0071301247771836,0.26024955436720143],
[0.344578313253012,0.0,0.00963855421686747,0.00963855421686747,0.09156626506024096,0.04096385542168675,0.0,0.004819277108433735,0.4987951807228916],
[0.44715447154471544,0.04878048780487805,0.032520325203252036,0.032520325203252036,0.0,0.16260162601626016,0.0,0.024390243902439025,0.25203252032520324],
[0.3333333333333333,0.034482758620689655,0.03495000467246052,0.09531819456125597,0.10727969348659004,0.11877394636015326,0.0,0.011494252873563218,0.26436781609195403],
[0.35978835978835977,0.03318903318903319,0.02873832994314922,0.05351275230793303,0.1091871091871092,0.1188071188071188,0.0,0.012987012987012988,0.2837902837902838],
[0.2935153583617747,0.034129692832764506,0.012798634812286689,0.004266211604095563,0.10921501706484642,0.12627986348122866,0.0,0.023890784982935155,0.39590443686006827],
[0.1774193548387097,0.03225806451612903,0.12273800157356411,0.19177812745869394,0.016129032258064516,0.14516129032258066,0.0,0.024193548387096774,0.2903225806451613],
[0.0,0.017964071856287425,0.21313305592205417,0.14614838120369433,0.0,0.11377245508982035,0.0,0.059880239520958084,0.4491017964071856],
[0.013201320132013201,0.0297029702970297,0.1416941694169417,0.22794279427942793,0.019801980198019802,0.1617161716171617,0.0,0.026402640264026403,0.3795379537953795],
[0.10628019323671498,0.0,0.06763285024154589,0.2463768115942029,0.043478260869565216,0.12560386473429952,0.0,0.028985507246376812,0.38164251207729466],
[0.3878787878787879,0.05656565656565657,0.0,0.0,0.16161616161616163,0.10707070707070707,0.0,0.00808080808080808,0.2787878787878788],
[0.15789473684210525,0.09210526315789473,0.10180055401662051,0.17451523545706368,0.039473684210526314,0.14473684210526316,0.0,0.02631578947368421,0.2631578947368421],
[0.6547945205479452,0.03287671232876712,0.005479452054794521,0.005479452054794521,0.0,0.07123287671232877,0.06027397260273973,0.005479452054794521,0.1643835616438356],
[0.4871382636655949,0.021543408360128618,0.06326917147513546,0.10779192177245296,0.01864951768488746,0.08520900321543408,0.01157556270096463,0.0038585209003215433,0.20096463022508038],
[0.7236842105263158,0.013157894736842105,0.02631578947368421,0.02631578947368421,0.0,0.09210526315789473,0.021929824561403508,0.0,0.09649122807017543],
[0.035897435897435895,0.020512820512820513,0.042971887550200816,0.5057460611677479,0.011965811965811967,0.06324786324786325,0.0,0.005128205128205128,0.3145299145299145],
[0.577717879604672,0.018867924528301886,0.009104522312069485,0.025037436358191074,0.03234501347708895,0.09164420485175202,0.014375561545372867,0.014375561545372867,0.2165318957771788],
[0.6570477247502775,0.01997780244173141,0.0,0.0,0.0,0.12652608213096558,0.0,0.004439511653718091,0.19200887902330743],
[0.45507527926177754,0.03885381253035454,0.01467487560091086,0.06983216665261027,0.008742107819329771,0.10344827586206896,0.009713453132588635,0.00825643516270034,0.2914035939776591],
[0.5344683226346637,0.026234998604521352,0.012753124321642323,0.02632027785530437,0.012001116382919341,0.08679877197878873,0.021769466927156013,0.008931063354730673,0.2707228579402735],
[0.03513909224011713,0.024890190336749635,0.05433927597393278,0.4405362730743835,0.021961932650073207,0.11273792093704246,0.0,0.01171303074670571,0.2986822840409956],
[0.02127659574468085,0.02695035460992908,0.06621846080835245,0.4500935959292362,0.019858156028368795,0.11063829787234042,0.0,0.011347517730496455,0.2936170212765957],
[0.6302083333333334,0.041666666666666664,0.0,0.0,0.0,0.109375,0.0,0.0,0.21875],
[0.6759493670886076,0.0,0.0,0.0,0.0,0.1341772151898734,0.0,0.0,0.189873417721519],
[0.2751322751322751,0.015873015873015872,0.0857276806643895,0.19645926818922602,0.024691358024691357,0.1128747795414462,0.0,0.007054673721340388,0.2821869488536155],
[0.6187050359712231,0.007194244604316547,0.0,0.0,0.0,0.1223021582733813,0.07434052757793765,0.0,0.1774580335731415],
[0.6823529411764706,0.0,0.0,0.00784313725490196,0.0,0.0784313725490196,0.09019607843137255,0.00392156862745098,0.13725490196078433],
[0.6794425087108014,0.0313588850174216,0.0,0.0,0.0,0.05226480836236934,0.0,0.0,0.23693379790940766],
[0.6518282988871225,0.0,0.0,0.04769475357710652,0.0,0.0397456279809221,0.08744038155802862,0.0,0.17329093799682035],
[0.017467248908296942,0.0,0.1325101641319078,0.5880138533353411,0.017467248908296942,0.03056768558951965,0.0,0.008733624454148471,0.2052401746724891],
[0.1339375629405841,0.025377643504531724,0.13931913639378085,0.1736718001621104,0.016918429003021148,0.10392749244712991,0.0,0.011278952668680765,0.3955689828801611],
[0.4509090909090909,0.03272727272727273,0.03234848484848485,0.04159090909090909,0.006060606060606061,0.11272727272727273,0.024242424242424242,0.02303030303030303,0.27636363636363637],
[0.38978723404255317,0.08085106382978724,0.026896828582898435,0.030124448012846246,0.002553191489361702,0.11319148936170213,0.02723404255319149,0.02553191489361702,0.3038297872340426],
[0.5672514619883041,0.029239766081871343,0.017543859649122806,0.017543859649122806,0.0,0.09064327485380116,0.029239766081871343,0.0,0.24853801169590642],
[0.6187624750499002,0.011976047904191617,0.017964071856287425,0.005988023952095809,0.0,0.08383233532934131,0.015968063872255488,0.007984031936127744,0.2375249500998004],
[0.42655935613682094,0.03118712273641851,0.042957746478873245,0.07977867203219316,0.03822937625754527,0.11066398390342053,0.005030181086519115,0.0030181086519114686,0.2625754527162978],
[0.4304388422035481,0.017740429505135387,0.04188712522045856,0.03747795414462081,0.04295051353874883,0.09897292250233426,0.0,0.014005602240896359,0.3165266106442577],
[0.0,0.06768734891216761,0.06560421406874994,0.4710597665920075,0.012087026591458501,0.11442385173247381,0.0,0.009669621273166801,0.2594681708299758],
[0.22541127056352817,0.035701785089254465,0.061228413533352725,0.13198124694967142,0.05320266013300665,0.1400070003500175,0.01400070003500175,0.04515225761288064,0.2933146657332867]]

ref_lstsq_ss = [[0.4240549828178694,0.032989690721649485,0.010996563573883162,0.005498281786941581,0.12096219931271478,0.10309278350515463,0.3024054982817869],
[0.27808676307007785,0.03225806451612903,0.02314884406096865,0.06250187896461533,0.0967741935483871,0.13236929922135707,0.37486095661846497],
[0.045454545454545456,0.0,0.11636363636363635,0.24727272727272728,0.03409090909090909,0.1590909090909091,0.3977272727272727],
[0.09224318658280922,0.025157232704402517,0.14538784067085953,0.1606918238993711,0.016771488469601678,0.12368972746331237,0.4360587002096436],
[0.07331975560081466,0.06109979633401222,0.1395924181275224,0.15979658390913748,0.020366598778004074,0.14867617107942974,0.3971486761710794],
[0.3112449799196787,0.07228915662650602,0.012048192771084338,0.008032128514056224,0.09437751004016064,0.12650602409638553,0.3755020080321285],
[0.10201834862385321,0.03547400611620795,0.07309909211446879,0.24274188647880338,0.047828746177370034,0.11314984709480122,0.3856880733944954],
[0.08552631578947369,0.06578947368421052,0.11816135734072021,0.3094702216066482,0.019736842105263157,0.07894736842105263,0.3223684210526316],
[0.7200992555831266,0.020843672456575684,0.0,0.0,0.0,0.07791563275434243,0.18114143920595535],
[0.5753424657534246,0.0,0.0273972602739726,0.0273972602739726,0.0,0.0821917808219178,0.2876712328767123],
[0.2742340532395781,0.04369663485685585,0.051839927874882945,0.11089236745409747,0.004018081366147665,0.12656956303365144,0.3887493721747865],
[0.08627450980392157,0.03431372549019608,0.1120471367354889,0.1771685495390209,0.052941176470588235,0.14215686274509803,0.39509803921568626],
[0.5720823798627003,0.02288329519450801,0.004576659038901602,0.009153318077803204,0.0,0.09839816933638444,0.2929061784897025],
[0.0,0.0379746835443038,0.07081575246132207,0.37644163150492266,0.012658227848101266,0.11814345991561181,0.3839662447257384],
[0.41346153846153844,0.0,0.0,0.0,0.0,0.17307692307692307,0.41346153846153844],
[0.0,0.07734806629834254,0.10162122996105423,0.26301965401684635,0.0718232044198895,0.08287292817679558,0.4033149171270718],
[0.028735632183908046,0.06321839080459771,0.12454966546577455,0.2547606793618117,0.08045977011494253,0.08045977011494253,0.367816091954023],
[0.025936599423631124,0.06340057636887608,0.11384337070615705,0.2665600875070994,0.07204610951008646,0.08069164265129683,0.37752161383285304],
[0.0,0.06358381502890173,0.10244553134726546,0.26749666518452647,0.06358381502890173,0.09826589595375723,0.4046242774566474],
[0.05714285714285714,0.06285714285714286,0.12190476190476192,0.2438095238095238,0.08,0.06285714285714286,0.37142857142857144],
[0.0,0.08092485549132948,0.09420403062021555,0.2930792063740041,0.09826589595375723,0.08670520231213873,0.3468208092485549],
[0.3920792079207921,0.047524752475247525,0.007920792079207921,0.015841584158415842,0.16237623762376238,0.08712871287128712,0.28712871287128716],
[0.3357664233576642,0.13138686131386862,0.0,0.0,0.18248175182481752,0.0948905109489051,0.25547445255474455],
[0.2548262548262548,0.03474903474903475,0.06446706446706446,0.15947115947115947,0.03861003861003861,0.08108108108108109,0.3667953667953668],
[0.05416666666666667,0.05416666666666667,0.13234374999999998,0.18848958333333335,0.020833333333333332,0.17083333333333334,0.37916666666666665],
[0.2633390705679862,0.07228915662650602,0.0402065404475043,0.08543889845094664,0.06712564543889846,0.12220309810671257,0.3493975903614458],
[0.4374240583232078,0.046172539489671933,0.04284970367247749,0.034914573362759445,0.07533414337788578,0.0959902794653706,0.267314702308627],
[0.3470790378006873,0.08247422680412371,0.01348136399682791,0.04493787998942639,0.12714776632302405,0.0859106529209622,0.29896907216494845],
[0.6695652173913044,0.0782608695652174,0.0,0.0,0.0,0.0817391304347826,0.17043478260869563],
[0.6861063464837049,0.030874785591766724,0.0,0.0,0.0,0.09433962264150944,0.18867924528301888],
[0.039423805913570885,0.03184230477634572,0.10851288219872005,0.3077115302349418,0.03411675511751327,0.10386656557998483,0.37452615617892343],
[0.2948905109489051,0.021897810218978103,0.0558809941790631,0.06674674304721427,0.05401459854014599,0.17226277372262774,0.3343065693430657],
[0.017429193899782137,0.026143790849673203,0.07722882064016368,0.3911807654164812,0.008714596949891068,0.10893246187363835,0.3703703703703704],
[0.6259541984732825,0.05343511450381679,0.0,0.0,0.0,0.03816793893129771,0.2824427480916031],
[0.3023255813953488,0.10077519379844961,0.024806201550387597,0.037209302325581395,0.0,0.24031007751937986,0.2945736434108527],
[0.7124183006535948,0.0,0.0,0.0,0.0,0.11764705882352941,0.16993464052287582],
[0.7417218543046358,0.039735099337748346,0.0,0.0,0.0,0.09271523178807947,0.12582781456953643],
[0.33126934984520123,0.018575851393188854,0.018575851393188854,0.018575851393188854,0.1393188854489164,0.10526315789473684,0.368421052631579],
[0.2951251646903821,0.023715415019762844,0.06736552672280463,0.20931431517442856,0.04743083003952569,0.12384716732542819,0.23320158102766797],
[0.25801749271137026,0.030612244897959183,0.052478134110787174,0.06559766763848396,0.048104956268221574,0.1457725947521866,0.39941690962099125],
[0.23113207547169812,0.02830188679245283,0.07004002287021155,0.09505431675242997,0.014150943396226415,0.1320754716981132,0.42924528301886794],
[0.008676789587852495,0.03253796095444685,0.07373704942167787,0.39047119352843057,0.008676789587852495,0.1193058568329718,0.36659436008676793],
[0.08806818181818182,0.04261363636363636,0.007575757575757577,0.0037878787878787876,0.3096590909090909,0.07954545454545454,0.46875],
[0.10810810810810811,0.0972972972972973,0.11183597390493942,0.2124883504193849,0.062162162162162166,0.11351351351351352,0.2945945945945946],
[0.3226381461675579,0.026737967914438502,0.04401480872069108,0.14671602906897024,0.06060606060606061,0.11764705882352941,0.2816399286987522],
[0.344578313253012,0.0,0.00963855421686747,0.00963855421686747,0.09156626506024096,0.04096385542168675,0.5036144578313253],
[0.44715447154471544,0.04878048780487805,0.032520325203252036,0.032520325203252036,0.0,0.16260162601626016,0.2764227642276423],
[0.3333333333333333,0.034482758620689655,0.03495000467246052,0.09531819456125597,0.10727969348659004,0.11877394636015326,0.27586206896551724],
[0.35978835978835977,0.03318903318903319,0.02873832994314922,0.05351275230793303,0.1091871091871092,0.1188071188071188,0.2967772967772968],
[0.2935153583617747,0.034129692832764506,0.012798634812286689,0.004266211604095563,0.10921501706484642,0.12627986348122866,0.41979522184300344],
[0.1774193548387097,0.03225806451612903,0.12273800157356411,0.19177812745869394,0.016129032258064516,0.14516129032258066,0.31451612903225806],
[0.0,0.017964071856287425,0.21313305592205417,0.14614838120369433,0.0,0.11377245508982035,0.5089820359281437],
[0.013201320132013201,0.0297029702970297,0.1416941694169417,0.22794279427942793,0.019801980198019802,0.1617161716171617,0.4059405940594059],
[0.10628019323671498,0.0,0.06763285024154589,0.2463768115942029,0.043478260869565216,0.12560386473429952,0.4106280193236715],
[0.3878787878787879,0.05656565656565657,0.0,0.0,0.16161616161616163,0.10707070707070707,0.2868686868686869],
[0.15789473684210525,0.09210526315789473,0.10180055401662051,0.17451523545706368,0.039473684210526314,0.14473684210526316,0.2894736842105263],
[0.6547945205479452,0.03287671232876712,0.005479452054794521,0.005479452054794521,0.0,0.07123287671232877,0.23013698630136986],
[0.4871382636655949,0.021543408360128618,0.06326917147513546,0.10779192177245296,0.01864951768488746,0.08520900321543408,0.21639871382636655],
[0.7236842105263158,0.013157894736842105,0.02631578947368421,0.02631578947368421,0.0,0.09210526315789473,0.11842105263157894],
[0.035897435897435895,0.020512820512820513,0.042971887550200816,0.5057460611677479,0.011965811965811967,0.06324786324786325,0.31965811965811963],
[0.577717879604672,0.018867924528301886,0.009104522312069485,0.025037436358191074,0.03234501347708895,0.09164420485175202,0.24528301886792453],
[0.6570477247502775,0.01997780244173141,0.0,0.0,0.0,0.12652608213096558,0.19644839067702552],
[0.45507527926177754,0.03885381253035454,0.01467487560091086,0.06983216665261027,0.008742107819329771,0.10344827586206896,0.30937348227294803],
[0.5344683226346637,0.026234998604521352,0.012753124321642323,0.02632027785530437,0.012001116382919341,0.08679877197878873,0.3014233882221602],
[0.03513909224011713,0.024890190336749635,0.05433927597393278,0.4405362730743835,0.021961932650073207,0.11273792093704246,0.3103953147877013],
[0.02127659574468085,0.02695035460992908,0.06621846080835245,0.4500935959292362,0.019858156028368795,0.11063829787234042,0.3049645390070922],
[0.6302083333333334,0.041666666666666664,0.0,0.0,0.0,0.109375,0.21875],
[0.6759493670886076,0.0,0.0,0.0,0.0,0.1341772151898734,0.189873417721519],
[0.2751322751322751,0.015873015873015872,0.0857276806643895,0.19645926818922602,0.024691358024691357,0.1128747795414462,0.2892416225749559],
[0.6187050359712231,0.007194244604316547,0.0,0.0,0.0,0.1223021582733813,0.2517985611510791],
[0.6823529411764706,0.0,0.0,0.00784313725490196,0.0,0.0784313725490196,0.23137254901960785],
[0.6794425087108014,0.0313588850174216,0.0,0.0,0.0,0.05226480836236934,0.23693379790940766],
[0.6518282988871225,0.0,0.0,0.04769475357710652,0.0,0.0397456279809221,0.26073131955484896],
[0.017467248908296942,0.0,0.1325101641319078,0.5880138533353411,0.017467248908296942,0.03056768558951965,0.21397379912663755],
[0.1339375629405841,0.025377643504531724,0.13931913639378085,0.1736718001621104,0.016918429003021148,0.10392749244712991,0.4068479355488419],
[0.4509090909090909,0.03272727272727273,0.03234848484848485,0.04159090909090909,0.006060606060606061,0.11272727272727273,0.3236363636363636],
[0.38978723404255317,0.08085106382978724,0.026896828582898435,0.030124448012846246,0.002553191489361702,0.11319148936170213,0.3565957446808511],
[0.5672514619883041,0.029239766081871343,0.017543859649122806,0.017543859649122806,0.0,0.09064327485380116,0.2777777777777778],
[0.6187624750499002,0.011976047904191617,0.017964071856287425,0.005988023952095809,0.0,0.08383233532934131,0.26147704590818366],
[0.42655935613682094,0.03118712273641851,0.042957746478873245,0.07977867203219316,0.03822937625754527,0.11066398390342053,0.27062374245472837],
[0.4304388422035481,0.017740429505135387,0.04188712522045856,0.03747795414462081,0.04295051353874883,0.09897292250233426,0.33053221288515405],
[0.0,0.06768734891216761,0.06560421406874994,0.4710597665920075,0.012087026591458501,0.11442385173247381,0.2691377921031426],
[0.22541127056352817,0.035701785089254465,0.061228413533352725,0.13198124694967142,0.05320266013300665,0.1400070003500175,0.3524676233811691]]

ref_names =[["1ado","CD0000001000.pcd"],
["1ed9","CD0000002000.pcd"],
["1hc9","CD0000004000.pcd"],
["5cha","CD0000005000.pcd"],
["2cga","CD0000006000.pcd"],
["1fa2","CD0000009000.pcd"],
["1bgl","CD0000010000.pcd"],
["1b8e","CD0000011000.pcd"],
["1ha7","CD0000012000.pcd"],
["1lin","CD0000013000.pcd"],
["1dgf","CD0000017000.pcd"],
["1kcw","CD0000018000.pcd"],
["2cts","CD0000019000.pcd"],
["1nls","CD0000020000.pcd"],
["1hrc","CD0000021000.pcd"],
["2bb2","CD0000022000.pcd"],
["4gcr","CD0000023000.pcd"],
["1elp","CD0000024000.pcd"],
["1m8u","CD0000025000.pcd"],
["1ha4","CD0000026000.pcd"],
["1hk0","CD0000027000.pcd"],
["1qfe","CD0000028000.pcd"],
["2dhq","CD0000029000.pcd"],
["3dni","CD0000030000.pcd"],
["3est","CD0000031000.pcd"],
["1cf3","CD0000033000.pcd"],
["1gpb","CD0000035000.pcd"],
["1bn6","CD0000036000.pcd"],
["1hda","CD0000037000.pcd"],
["1n5u","CD0000038000.pcd"],
["1igt","CD0000039000.pcd"],
["1blf","CD0000042000.pcd"],
["1les","CD0000043000.pcd"],
["1ax8","CD0000044000.pcd"],
["193l","CD0000045000.pcd"],
["1ymb","CD0000047000.pcd"],
["1a6m","CD0000048000.pcd"],
["1k6j","CD0000049000.pcd"],
["1ova","CD0000050000.pcd"],
["1dot","CD0000051000.pcd"],
["1ppn","CD0000052000.pcd"],
["1ofs","CD0000053000.pcd"],
["1air","CD0000054000.pcd"],
["2psg","CD0000055000.pcd"],
["3pmg","CD0000057000.pcd"],
["3pgk","CD0000058000.pcd"],
["1une","CD0000059000.pcd"],
["1hnn","CD0000060000.pcd"],
["1a49","CD0000061000.pcd"],
["1rhs","CD0000062000.pcd"],
["3rn3","CD0000063000.pcd"],
["1ba7","CD0000065000.pcd"],
["1cbj","CD0000068000.pcd"],
["1thw","CD0000069000.pcd"],
["7tim","CD0000070000.pcd"],
["1ubi","CD0000071000.pcd"],
["2nop","CD0000099000.pcd"],
["2gif","CD0000100000.pcd"],
["1qhj","CD0000101000.pcd"],
["1nqh","CD0000102000.pcd"],
["1l7v","CD0000103000.pcd"],
["1kpk","CD0000104000.pcd"],
["1be3","CD0000105000.pcd"],
["2dyr","CD0000106000.pcd"],
["1fep","CD0000107000.pcd"],
["1fcp","CD0000108000.pcd"],
["2nr9","CD0000109000.pcd"],
["1j95","CD0000110000.pcd"],
["1xl4","CD0000111000.pcd"],
["2cfq","CD0000112000.pcd"],
["2a65","CD0000113000.pcd"],
["1nkz","CD0000114000.pcd"],
["2oar","CD0000115000.pcd"],
["2vdf","CD0000119000.pcd"],
["3jqo","CD0000120000.pcd"],
["1pcr","CD0000121000.pcd"],
["2wjn","CD0000122000.pcd"],
["1hzx","CD0000123000.pcd"],
["1rh5","CD0000124000.pcd"],
["1t5s","CD0000125000.pcd"],
["1nek","CD0000126000.pcd"],
["1a0s","CD0000127000.pcd"],
["2j58","CD0000128000.pcd"]]

In [9]:
#@title Load all PDBMD2CD modules { vertical-output: true, display-mode: "form" }
import re
import string
import os
import time
import numpy as np
import sys
import math
import random
from math import sqrt
from scipy.spatial.distance import cdist, euclidean, pdist, squareform
from scipy.optimize import lsq_linear, minimize
from itertools import groupby, repeat

from IPython.display import HTML, display
import plotly.express as px

import argparse
#import requests
import time
import datetime

import tarfile
import zipfile
from shutil import make_archive, move, copy

import networkx as nx
import json
from collections import defaultdict

from Bio.PDB import PDBList, PDBIO, PDBParser, MMCIFParser

from Bio.PDB.mmcifio import MMCIFIO
from Bio.PDB.MMCIF2Dict import MMCIF2Dict
import subprocess

#import config

twistcutoff=[-10,59]
an=[35, 110]
no_cand=12


# Path to write log files to
LOGPATH = "./out/pdbmd2cd.log"

# Path to DSSP executable on your system, or command to run DSSP
DSSPPATH = "mkdssp"


def progress(value, max=100):
	return HTML("""
		<progress
			value='{value}'
			max='{max}',
			style='width: 100%'
		>
			{value}
		</progress>
	""".format(value=value, max=max))



def extract_file(path, to_directory='.'):
	
	if path.endswith('.zip'):
		opener, mode = zipfile.ZipFile, 'r'
	elif path.endswith('.tar.gz') or path.endswith('.tgz'):
		opener, mode = tarfile.open, 'r:gz'
	elif path.endswith('.tar.bz2') or path.endswith('.tbz'):
		opener, mode = tarfile.open, 'r:bz2'
	else: 
		raise ValueError("Could not extract `%s` as no appropriate extractor is found" % path)
	
	cwd = os.getcwd()
	os.chdir(to_directory)
	
	try:
		file = opener(path, mode)
		try: file.extractall()
		finally: file.close()
	finally:
		os.chdir(cwd)


def pdbfile_to_dssp_local(directory, filename):
	
	subprocess.call([DSSPPATH, "-i", os.path.join(directory,str(filename)), "-o", os.path.join(directory,str(filename)+".dssp")])

	try:
		with open(os.path.join(directory, str(filename)+".dssp"), "r") as dssp_in:
			return(dssp_in.readlines(), True)
	except FileNotFoundError:
		failed.append(filename)
		return("", False)



def angleTwoLines(a1, b1, c1, a2, b2, c2):
	
	numerator=a1*a2+b1*b2+c1*c2
	denom=sqrt(a1**2+b1**2+c1**2)*sqrt(a2**2+b2**2+c2**2)
	costheta=numerator/denom
	angle=np.arccos(costheta)
	return(angle)

class DSSPData:

	def __init__(self):
			"""Summary
			"""
			self.num        = []
			self.resnum = []
			self.moltyp = []
			self.aa   = []
			self.struct = []
			self.bp1        = []
			self.bp2        = []
			self.sheet      = []
			self.acc        = []
			self.h_nho1 = []
			self.h_ohn1 = []
			self.h_nho2 = []
			self.h_ohn2 = []
			self.tco        = []
			self.kappa  = []
			self.alpha  = []
			self.phi        = []
			self.psi        = []
			self.xyzca      = []
			self.antipar = []
			

	def parseDSSP(self, input_handle):
		
		#input_handle = open(file, 'r')

		line_num = 0
		start=False
		for line in input_handle:
	
			if( re.search('#', line) ):
				start=True
				continue

			if( start and line[12:14].strip()!="!"):
				self.num.append(        line[0:5].strip() )
				self.resnum.append( line[5:10].strip() )
				self.moltyp.append( line[10:12].strip() )
				self.aa.append(   line[12:14].strip() )
				self.struct.append( line[16:17].strip() )
				self.antipar.append( line[23:25] )
				self.bp1.append(        line[25:29].strip() )
				self.bp2.append(        line[29:33].strip() )
				self.sheet.append(      line[33].strip() )
				self.acc.append(        line[34:38].strip() )
				self.h_nho1.append( line[38:50].strip() )
				self.h_ohn1.append( line[50:61].strip() )
				self.h_nho2.append( line[61:72].strip() )
				self.h_ohn2.append( line[72:83].strip() )
				self.tco.append(        line[83:91].strip() )
				self.kappa.append(  line[91:97].strip() )
				self.alpha.append(  line[97:103].strip() )
				self.phi.append(        line[103:109].strip() )
				self.psi.append(        line[109:115].strip() )
				self.xyzca.append(      [float(line[115:122].strip()),float(line[122:129].strip()),float(line[129:136].strip())] )
				
		#input_handle.close()
	def getNums(self):
		
		return self.num
	def getResnums(self):
		
		return self.resnum
	def getChainType(self):
		
		return self.moltyp
	def getAAs(self):
		
		return self.aa
	def getSecStruc(self):
		
		return self.struct
	def getAntiPar(self):
		
		return self.antipar
	def getBP1(self):
		
		return self.bp1
	def getBP2(self):
		
		return self.bp2
	def getSheet(self):
		
		return self.sheet
	def getACC(self):
		
		return self.acc
	def getH_NHO1(self):
		
		return self.h_nho1
	def getH_NHO2(self):
		
		return self.h_nho2
	def getH_OHN1(self):
		
		return self.h_ohn1
	def getH_OHN2(self):
		
		return self.h_ohn2
	def getTCO(self):
		
		return self.tco
	def getKAPPA(self):
		
		return self.kappa
	def getALPHA(self):
		
		return self.alpha
	def getPHI(self):
		
		return self.phi
	def getPSI(self):
		
		return self.psi
	def getXYZ(self):
		
		return self.xyzca

def refinementCOMBO(cd, scores):
	
	flag=0

	while(flag==0):
		cd = np.asarray(cd)
		scores=scores
		meancd= np.mean(cd, axis=0)
		stdcd=np.std(cd, axis=0)
		newcd=[]
		newscores=[]
		newq=[]
		qlist=list(range(0, len(cd)))
		for x,s,q in zip(cd, scores, qlist):
			bad=0
			for idx,y in enumerate(x):
				if(np.abs(y)>(np.abs(meancd[idx])+(1.5*stdcd[idx]))):
					bad+=1
			if(bad/len(cd[0]) <= (1./6.)):
				newcd.append(x)
				newscores.append(s)
				newq.append(q)
		if(len(newcd)==len(cd)):
			cd=np.asarray(newcd)
			scores=newscores
			qlist=newq
			flag=1
		else:
			cd=np.asarray(newcd)
			scores=newscores
			qlist=newq

	return(cd, scores, qlist)

def euclidDist(x, y):
	
	distance=sqrt(sum((x-y)**2))
	return(distance)
	 
def unit_vector(vector):
	
	return vector / np.linalg.norm(vector)

def angle_between(v1, v2):
	"""Returns the angle in degrees between vectors 'v1' and 'v2'::
	
	>>> angle_between((1, 0, 0), (0, 1, 0))
	1.5707963267948966
	>>> angle_between((1, 0, 0), (1, 0, 0))
	0.0
	>>> angle_between((1, 0, 0), (-1, 0, 0))
	3.141592653589793
	
	
	"""
	v1_u = unit_vector(v1)
	v2_u = unit_vector(v2)
	return math.degrees(np.arccos(np.clip(np.dot(v1_u, v2_u), -1.0, 1.0)))

def get_vector(p1, p2):
	
	v=p1-p2

	return v

def distance_points(p1, v1, p2, v2):
	"""Returns close to the minimum distance between 2 lines that define secondary structure elements
	by using parametric rep of line and iterating t between 0 and 1 incrementing by 0.01.
	create array for each line with points alone line. Find distance between points. Take minimum 
	
	
	"""
	l1=[]
	l2=[]
	for i in range(0, 100):
		t=i*0.01

		# make array of l1 points
		l1.append(p1-v1*t)
		# make array of l2 points
		l2.append(p2-v2*t)

	l1=np.asarray(l1)
	l2=np.asarray(l2)

	mindistance=np.min(cdist(l1,l2, 'euclidean'))

	return(mindistance)

def distance_points_max(p1, v1, p2, v2):
	
	l1=[]
	l2=[]
	for i in range(0, 100):
		t=i*0.01

		# make array of l1 points
		l1.append(p1-v1*t)
		# make array of l2 points
		l2.append(p2-v2*t)

	l1=np.asarray(l1)
	l2=np.asarray(l2)

	mindistance=np.max(cdist(l1,l2, 'euclidean'))

	return(mindistance)

def svd_line(data):
	
	datamean = data.mean(axis=0)
	uu, dd, vv = np.linalg.svd(data - datamean)
	linepts = vv[0] * np.mgrid[-7:7:2j][:, np.newaxis]
	linepts += datamean

	return(linepts)

def rmsd(exp, pred):
	
	total=0
	for i in range (0, len(pred)):
		x=(pred[i]-exp[i])**2
		total+=x
	result=sqrt(total/len(pred))

	return(result)

def getSS_multi(pdbfiles, directory, twistcutoff, mode, an):

	#set up progress bar
	out = display(progress(0, 100), display_id=True)
	
	
	all_ss=[]
	lstsq_ss=[]
	dssp_files=[]
	failed=[]
	all_HEO_full={}

	missing_dict={}
	# need to add this in

	with open(os.path.join(directory, "missing.txt"), "w") as missing_out:
		missing_out.write("")


	for i in range(0, len(pdbfiles)):
		
		try:
			
			f=pdbfiles[i]

			missing=0
			
			if(f in missing_dict):
				missing+=missing_dict[f]
				""" mmdict=MMCIF2Dict(os.path.join(outdir, pc+".cif"))
				if("_pdbx_unobs_or_zero_occ_residues.auth_seq_id" in mmdict):
					missing+=len(mmdict["_pdbx_unobs_or_zero_occ_residues.auth_seq_id"]) """
			else:
				with open(os.path.join(directory, f)) as fin:
					for l in fin.readlines():
						l=l.strip()
						missing_bool=re.search("^REMARK\s465\s+[A-Z]{3}\s[A-Z]\s*", l)
						if(missing_bool):
							missing+=1
			with open(os.path.join(directory, "missing.txt"), "a") as missing_out:
				missing_out.write(str(f) +" "+str(missing)+"\n")

			if(mode=="file"):
				dssp_input,success=pdbfile_to_dssp_local(directory,f)
				dssp_files.append(f)
			
			if(success==True):
				dd_ob=DSSPData()
				dd_ob.parseDSSP(dssp_input)
				cur_ch=99999
				heo_full=[]
				heo_ch=[]
				A=0 #right
				P=0
				H=0
				I=0
				G=0
				B=0
				T=0
				O=missing
				l=len(dd_ob.getSecStruc())+missing
				if(l==0):
					sys.exit()
				for ss, aorp, phi,ch in zip(dd_ob.getSecStruc(), dd_ob.getAntiPar(), dd_ob.getPHI(), dd_ob.getChainType()):
					if(ch!=cur_ch):
						cur_ch=ch
						if(len(heo_ch)>0):
							heo_full.append(heo_ch)
						heo_ch=[]

					if(ss=="H"):
						H+=1
						heo_ch.append("H")
					elif(ss=="E"):
						if(aorp[0].isupper() or aorp[1].isupper()):
							A+=1
						else:
							P+=1
						heo_ch.append("E")
					elif(ss=="I"):
						I+=1
						heo_ch.append("C")
					elif(ss=="G"):
						G+=1
						heo_ch.append("H")
					elif(ss=="B"):
						B+=1
						heo_ch.append("C")
					elif(ss=="T"):
						T+=1
						heo_ch.append("C")
					else:
						O+=1
						heo_ch.append("C")

				perp_count=getETOP_hbond(dd_ob, an)

				# for areas, name in zip(areas_all, name_all):
				#   if(name!=False):
				#       a_dists.append([areas])
				#       names.append(name)
				heo_full.append(heo_ch)
				all_HEO_full[f]=(heo_full)
				A2=A*perp_count
				A1=A-A2
				
				all_ss.append([(H/l),G/l,A1/l,A2/l,P/l,T/l,I/l,B/l,O/l])
				lstsq_ss.append([(H/l),G/l,A1/l,A2/l,P/l,T/l,I/l+B/l+O/l])

				percentage=str(50+int(i/len(pdbfiles)*50))

			else:
				failed.append(pdbfiles[i])
				percentage=str(50+int(i/len(pdbfiles)*50))

				
		except:
			failed.append(pdbfiles[i])
			percentage=str(50+int(i/len(pdbfiles)*50))

		out.update(progress(int((i/len(pdbfiles)*100)), 100))
			
		
	return(all_ss, lstsq_ss, dssp_files, failed, all_HEO_full)


def getETOP_hbond(dd_ob, an):
	"""
	curves following h-bonding patterns of beta residues
	
	
	"""
	ss=np.asarray(dd_ob.getSecStruc())
	xyz=np.asarray(dd_ob.getXYZ())
	res=np.asarray(dd_ob.getNums())
	ap=np.asarray(dd_ob.getAntiPar())
	bp1=np.asarray(dd_ob.getBP1())
	bp2=np.asarray(dd_ob.getBP2())
	sheet=np.asarray(dd_ob.getSheet())

	insheet="INIT"
	strands={}
	strand=[]
	strand_counter=0
	total_APE=0

	# make dictionary of resnum to xyz

	num_xyz_dict = dict(zip(res, xyz))


	# graph rep of h-bonded backbone vectors
	h_g = nx.Graph()
	

	for i in range(0, len(ss)):
		if(ss[i]=="E"):
			
			if(ap[i][0].isupper() or ap[i][1].isupper()):
				if(i<len(ss)-1):
					if(ss[i+1]=="E"):

						a1=num_xyz_dict[res[i]][0]-num_xyz_dict[res[i+1]][0]
						b1=num_xyz_dict[res[i]][1]-num_xyz_dict[res[i+1]][1]
						c1=num_xyz_dict[res[i]][2]-num_xyz_dict[res[i+1]][2]
						nodename=[a1,b1,c1]
						if(bp1[i] !="0"):
							a2=num_xyz_dict[bp1[i]][0]-num_xyz_dict[str(int(bp1[i])+1)][0]
							b2=num_xyz_dict[bp1[i]][1]-num_xyz_dict[str(int(bp1[i])+1)][1]
							c2=num_xyz_dict[bp1[i]][2]-num_xyz_dict[str(int(bp1[i])+1)][2]
							nodename2=[a2,b2,c2]
							h_g.add_node(json.dumps(nodename))
							h_g.add_node(json.dumps(nodename2))
							h_g.add_edge(json.dumps(nodename2),json.dumps(nodename))
						
						if(bp2[i] !="0"):
							a2=num_xyz_dict[bp2[i]][0]-num_xyz_dict[str(int(bp2[i])-1)][0]
							b2=num_xyz_dict[bp2[i]][1]-num_xyz_dict[str(int(bp2[i])-1)][1]
							c2=num_xyz_dict[bp2[i]][2]-num_xyz_dict[str(int(bp2[i])-1)][2]
							nodename2=[a2,b2,c2]
							h_g.add_node(json.dumps(nodename))
							h_g.add_node(json.dumps(nodename2))
							h_g.add_edge(json.dumps(nodename2),json.dumps(nodename))
						
						
	edge_list=list(h_g.edges())             
								
	A2=0
	
	for e in edge_list:
		e1=json.loads(e[0])
		e2=json.loads(e[1])
		a1=e1[0]
		b1=e1[1]
		c1=e1[2]
		a2=e2[0]
		b2=e2[1]
		c2=e2[2]
		angle=angleTwoLines(a1,b1,c1,a2,b2,c2)
		angle=math.degrees(angle)
		if((angle > int(an[1]) or angle < int(an[0]))):
			A2+=1



	if(len(edge_list)>0):
		A2=A2/len(edge_list)
	else:
		A2=0



	return(A2)

def isNMR(file, directory):
	
	if(file.endswith(".pdb") or file.endswith(".ent") ):
		p = PDBParser(QUIET=True)
		structure = p.get_structure("X", os.path.join(directory, file))
		if(len(structure)>1):
			nmr=True
			new_pdb_files=[]
			for i, model in enumerate(structure):
				name=file[:-4]+"_"+str(i)+".pdb"
				io=PDBIO()
				io.set_structure(model)
				io.save(os.path.join(directory, name))
				new_pdb_files.append(name)
			#pdb_files.remove(file)
			#pdb_files=pdb_files+new_pdb_files
		else:
			new_pdb_files=[file]
	elif(file.endswith(".cif")):
		p = MMCIFParser(QUIET=True)
		structure = p.get_structure("X", os.path.join(directory, file))
		if(len(structure)>1):
			nmr=True
			new_pdb_files=[]
			for i, model in enumerate(structure):
				name=file[:-4]+"_"+str(i)+".pdb"
				io=PDBIO()
				io.set_structure(model)
				io.save(os.path.join(directory, name))
				new_pdb_files.append(name)
				if(file in missing_dict):
					missing_dict[name]=missing_dict[file]
			#pdb_files.remove(file)
			#pdb_files=pdb_files+new_pdb_files
		else:
			new_pdb_files=[file]

	return(new_pdb_files)

"""
WORK STARTS HERE 

"""

"""
get all ref lstsq_ss and lincomb_ss from reference files

"""

dsspfile_refs=[]
pcdfiles_refs=[]

for line in ref_names:
	
	dsspfile_refs.append(line[0].strip())
	pcdfiles_refs.append(line[1].strip())
	
ref_lstsq_ss=np.asarray(ref_lstsq_ss)
ref_lincomb_ss=np.asarray(ref_lincomb_ss)
ref_cd=np.asarray(ref_cd)
ref_basis=np.asarray(ref_basis)


# functions for different modes

def makePred(f, lc_ss, lq_ss):
	
	# lincomb first

	candidates=[]

	for idx, ref_ss in enumerate(ref_lincomb_ss):
		candidates.append(euclidean(ref_ss, lc_ss))

	candidates_sorted, candidate_idxs_sorted, ref_lincomb_ss_sorted = zip(*[(x, y,z) for x, y,z in sorted(zip(candidates, list(range(0, len(candidates))), ref_lincomb_ss))])

	# get refs closest in ss to the query - caveat up to half more alpha/beta, vice versa. if < half greater then remainder is less.

	ss_greater=[]
	idx_greater=[]
	ss_lesser=[]
	idx_lesser=[]

	# get largest secondary structure proportion out of H,a1,a2,a3,P

	main_ss=np.argmax(np.asarray(lc_ss)[0:-1])

	for ss_sort, idxs_sorted in zip(ref_lincomb_ss_sorted, candidate_idxs_sorted):
		if(lc_ss[main_ss]< ss_sort[main_ss]):
			ss_greater.append(ss_sort)
			idx_greater.append(idxs_sorted)
		else:
			ss_lesser.append(ss_sort)
			idx_lesser.append(idxs_sorted)

	if(len(ss_greater)>no_cand/2):
		greater_no=int(no_cand/2)
	else:
		greater_no=int(len(ss_greater))
	if(len(idx_lesser) >= int(no_cand-greater_no)):
		lesser_no=int(no_cand-greater_no)
	else:
		lesser_no=len(idx_lesser)
		greater_no=int(no_cand-len(idx_lesser))
	

	if(greater_no ==0):
		candidate_ss=ref_lincomb_ss[np.asarray(idx_lesser[0:lesser_no])]
		candidate_cd=ref_cd[np.asarray(idx_lesser[0:lesser_no])]
	elif(lesser_no ==0):
		candidate_ss=ref_lincomb_ss[np.asarray(idx_greater[0:greater_no])]
		candidate_cd=ref_cd[np.asarray(idx_greater[0:greater_no])]

	else:
		candidate_ss=np.asarray(ref_lincomb_ss[np.asarray(idx_greater[0:greater_no])].tolist()+ref_lincomb_ss[np.asarray(idx_lesser[0:lesser_no])].tolist())
		candidate_cd=np.asarray(ref_cd[np.asarray(idx_greater[0:greater_no])].tolist()+ref_cd[np.asarray(idx_lesser[0:lesser_no])].tolist())

	# now need to minimise obj func to obtain proportions of each ref to get spectra
	objFun= lambda p: (euclidean(np.asarray(lc_ss), (np.sum(np.asarray([p]*len(candidate_ss.T)).T*candidate_ss, axis=0))))
	bnds = list(repeat([0.0, 1.0],len(candidate_ss)))
	x0 = list(repeat(0.1,len(candidate_ss)))
	X=minimize(objFun, x0, bounds=bnds)

	prediction=[]

	for x, cand_cd in zip(X.x, candidate_cd):
		
		prediction.append(cand_cd*x)

	lc_pred=np.sum(np.asarray(prediction), axis=0)

	# Lstsq second

	lq_pred=np.sum(ref_basis*np.asarray(lq_ss), axis=1)

	# combo pred third

	cd_refined_pred2, scores_refined_pred2, q_refined_pred2=refinementCOMBO(np.asarray([lq_pred, lc_pred]), list(range(0, 2)))
	if(len(cd_refined_pred2)>1):
		combo_pred=np.mean(np.asarray(cd_refined_pred2), axis=0)
	else:
		combo_pred=cd_refined_pred2[0]

	return(combo_pred)


def writeOutput(pred, f, outdir, success):
	if(success):
		output_f = os.path.join(outdir, str(f)+"_cd.txt")
		curr_time = str(datetime.datetime.now().strftime("%y-%m-%d, %H:%M:%S"))
		with open(output_f, "w") as fout:
			fout.write("# PDBMD2CD Standalone\n")
			fout.write("# E.D. Drew and R.W. Janes\n")
			fout.write("# https://doi.org/10.1093/nar/gkaa296\n")
			fout.write("# "+curr_time+"\n")
			fout.write("# Wavelength (nm)\tPredicted CD signal (delta epsilon)\n")
			for i, p in enumerate(pred):
				fout.write(str(260-i)+"\t"+str(p)+"\n")
	else:
		output_f = os.path.join(outdir, str(f)+"_cd.txt")
		curr_time = str(datetime.datetime.now().strftime("%y-%m-%d, %H:%M:%S"))
		with open(output_f, "w") as fout:
			fout.write("# PDBMD2CD Standalone\n")
			fout.write("# E.D. Drew and R.W. Janes\n")
			fout.write("# https://doi.org/10.1093/nar/gkaa296\n")
			fout.write("# "+curr_time+"\n")
			fout.write("# Wavelength (nm)\tPredicted CD signal (delta epsilon)\n")
			fout.write("Calculation Failed\n")
			

def writeLog(nFiles, nFails, failed, logpath):
	curr_time=str(datetime.datetime.now().strftime("%y-%m-%d, %H:%M:%S"))
	with open(logpath, "w") as fout:
		fout.write("PDBMD2CD - Colab Version"+"\n")
		fout.write("Time: "+curr_time+"\n")
		fout.write("Number of Files: "+str(nFiles)+"\n")
		fout.write("Number of Failures: "+str(nFails)+"\n")
		fout.write("Failures:"+"\n")
		for fail in failed:
			fout.write(str(fail)+"\n")


def run_colab():
	
	indir = "/content/in"
	outdir = "/content/out"

	print("Running DSSP....\n")
	all_files = []
	for f in os.listdir(indir):
		if(f.endswith(".pdb")):
			# check NMR/Models here
			files = isNMR(f, indir)
			all_files+=files

	all_lincomb_ss,all_lstsq_ss, dssp_files, failed, all_HEO_full = getSS_multi(all_files, indir, twistcutoff, "file", an)

	print("{0} failed calculations, {1} successful\n".format(str(len(failed)), str(len(dssp_files))))
	
	for f in failed:
		writeOutput([], f, outdir, False) # unecessary but just in case

	print("Making Predictions....\n")

	out2 = display(progress(0, 100), display_id=True)
	
	all_preds = []
	count=0
	for f,  lc_ss, lstsq_ss in zip(dssp_files, all_lincomb_ss, all_lstsq_ss):
		count+=1
		if(f in failed):
			writeOutput([], f, outdir, False) # unecessary but just in case
		else:
			combo_pred = makePred(f, lc_ss, lstsq_ss)
			all_preds.append(combo_pred)

		writeOutput(combo_pred, f, outdir, True)

		out2.update(progress(int((count/len(dssp_files)*100)), 100))

	mean_pred = np.mean(np.asarray(all_preds), axis=0)

	writeOutput(mean_pred, "average", outdir, True)
	
	writeLog(len(failed), len(combo_pred), failed, LOGPATH)
	print("Log written: "+LOGPATH)
	print("Please remember to send log file to r.w.janes@qmul.ac.uk periodically so we can track usage. Thanks!")
	print("Done!")

	return(all_preds, mean_pred, dssp_files, failed)

In [10]:
#@title Run the predictions! { vertical-output: true, display-mode: "form" }
all_preds, mean_pred, dssp_files, failed = run_colab()

Running DSSP....



1 failed calculations, 22 successful

Making Predictions....



Log written: ./out/pdbmd2cd.log
Please remember to send log file to r.w.janes@qmul.ac.uk periodically so we can track usage. Thanks!
Done!


In [24]:
len(all_preds)

22

In [35]:
#@title Plot results { vertical-output: true, display-mode: "form" }
import plotly.graph_objects as go

good_files = []
for f in dssp_files:
  if f not in failed:
    good_files.append(f)

data = {
    "Wavelength (nm)":list(range(180, 261))[::-1], 
    "CD signal (dE)":mean_pred,
    
}
fig = px.line(data_frame = data, x="Wavelength (nm)", y="CD signal (dE)")
for i in range(0,len(all_preds)):
  fig.add_traces(go.Scatter(x=data["Wavelength (nm)"], 
                            y=all_preds[i], 
                            text = [good_files[i]]*len(all_preds[i]),
                            hovertemplate =
                          '<b>%{text}</b><br>'+
                          'CD signal: $%{y:.2f} dE'+
                          '<br>Wavelength: %{x} nm<extra></extra>',
                            mode='lines', 
                            line = dict(color = "lightblue")))
fig.update_yaxes(zeroline=True,zerolinewidth=2, zerolinecolor="grey")
fig.data = fig.data[::-1]
fig.update_layout(showlegend=False)
fig.show()

In [ ]:
#@title Zip up results { vertical-output: true, display-mode: "form" }
!zip results.zip ./out/*


  adding: out/2m6q_0.pdb_cd.txt (deflated 47%)
  adding: out/2m6q_1.pdb_cd.txt (deflated 46%)
  adding: out/2m6q_2.pdb_cd.txt (deflated 46%)
  adding: out/2m6q_3.pdb_cd.txt (deflated 46%)
  adding: out/2m6q_4.pdb_cd.txt (deflated 47%)
  adding: out/2m6q_5.pdb_cd.txt (deflated 46%)
  adding: out/2m6q_6.pdb_cd.txt (deflated 47%)
  adding: out/2m6q_7.pdb_cd.txt (deflated 46%)
  adding: out/2m6q_8.pdb_cd.txt (deflated 47%)
  adding: out/2m6q_9.pdb_cd.txt (deflated 46%)
  adding: out/2y65.pdb_cd.txt (deflated 46%)
  adding: out/329d.pdb_cd.txt (deflated 13%)
  adding: out/5h04.pdb_cd.txt (deflated 47%)
  adding: out/average_cd.txt (deflated 46%)
  adding: out/pdbmd2cd.log (deflated 16%)


In [ ]:
#@title Download results.zip { display-mode: "form" }
# here we do the download
from google.colab import files
files.download('results.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>